In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Zonotope Containment"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import pypolycontain as pp\n",
    "import time\n",
    "import pydrake.solvers.mathematicalprogram as MP\n",
    "# use Gurobi solver\n",
    "import pydrake.solvers.gurobi as Gurobi_drake\n",
    "gurobi_solver=Gurobi_drake.GurobiSolver()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "def alpha_necessary_old(Z_i,Z_o):\n",
    "    program=MP.MathematicalProgram()\n",
    "    zeta=program.NewContinuousVariables(Z_o.G.shape[1],2**Z_i.G.shape[1],\"zeta\")\n",
    "    beta=program.NewContinuousVariables(1,\"beta\")\n",
    "    V=pp.vcube(Z_i.G.shape[1])\n",
    "    for i in range(V.shape[0]):\n",
    "        program.AddLinearEqualityConstraint( Aeq=Z_o.G, beq=np.dot(Z_i.G,V.T)[:,i], vars= zeta[:,i]  )\n",
    "    program.AddLinearConstraint(  np.less_equal(zeta,beta*np.ones(zeta.shape),dtype='object').flatten() )\n",
    "    program.AddLinearConstraint(  np.less_equal(-zeta,beta*np.ones(zeta.shape),dtype='object').flatten() )\n",
    "    program.AddLinearCost(np.eye(1),np.zeros((1)),beta)\n",
    "    result=gurobi_solver.Solve(program,None,None)\n",
    "    if result.is_success():\n",
    "        alpha=1/result.GetSolution(beta)[0]\n",
    "        return alpha\n",
    "    else:\n",
    "        print(\"optimization failed\")\n",
    "        \n",
    "def alpha_necessary_older(Z_i,Z_o):\n",
    "    alpha_min=np.inf\n",
    "    V=pp.vcube(Z_i.G.shape[1])\n",
    "    B=np.dot(Z_i.G,V.T)\n",
    "    for i in range(V.shape[0]):\n",
    "        program=MP.MathematicalProgram()\n",
    "        zeta=program.NewContinuousVariables(Z_o.G.shape[1],\"zeta\")\n",
    "        beta=program.NewContinuousVariables(1,\"beta\")\n",
    "        program.AddLinearEqualityConstraint( Aeq=Z_o.G, beq=B[:,i:i+1], vars= zeta  )\n",
    "#         for j in range(zeta.shape[0]):\n",
    "#             program.AddLinearConstraint(  zeta[j]-beta, -np.inf, 0 )\n",
    "#             program.AddLinearConstraint(  -zeta[j]+ beta, -np.inf, 0 )\n",
    "        program.AddLinearConstraint(  np.less_equal(zeta,beta*np.ones(zeta.shape),dtype='object').flatten() )\n",
    "        program.AddLinearConstraint(  np.less_equal(-zeta,beta*np.ones(zeta.shape),dtype='object').flatten() )\n",
    "        program.AddLinearCost(np.eye(1),np.zeros((1)),beta)\n",
    "        result=gurobi_solver.Solve(program,None,None)\n",
    "        if result.is_success():\n",
    "            alpha=1/result.GetSolution(beta)[0]\n",
    "            alpha_min=min(alpha,alpha_min)\n",
    "        else:\n",
    "            print(\"optimization failed\")\n",
    "    return alpha_min\n",
    "\n",
    "def alpha_necessary(Z_i,Z_o):\n",
    "    alpha_min=np.inf\n",
    "    V=pp.vcube(Z_i.G.shape[1])\n",
    "    B=np.dot(Z_i.G,V.T)\n",
    "    for i in range(V.shape[0]):\n",
    "        program=MP.MathematicalProgram()\n",
    "        zeta=program.NewContinuousVariables(Z_o.G.shape[1],\"zeta\")\n",
    "        alpha=program.NewContinuousVariables(1,\"alpha\")\n",
    "        Aeq=np.hstack(( Z_o.G, -B[:,i:i+1]))\n",
    "        program.AddLinearEqualityConstraint( Aeq=np.hstack(( Z_o.G, -B[:,i:i+1])), \\\n",
    "                                            beq=np.zeros((Z_o.G.shape[0])),\\\n",
    "                                            vars= np.hstack((zeta,alpha))  )\n",
    "        program.AddBoundingBoxConstraint(-1,1,zeta)\n",
    "        program.AddLinearCost(-np.eye(1),np.zeros((1)),alpha)\n",
    "        result=gurobi_solver.Solve(program,None,None)\n",
    "        if result.is_success():\n",
    "            alpha=result.GetSolution(alpha)[0]\n",
    "            alpha_min=min(alpha,alpha_min)\n",
    "        else:\n",
    "            print(\"optimization failed\")\n",
    "    return alpha_min\n",
    "        \n",
    "        \n",
    "def alpha_sufficient(Z_i,Z_o):\n",
    "    program=MP.MathematicalProgram()\n",
    "    beta=program.NewContinuousVariables(1,\"beta\")\n",
    "    circumbody=pp.to_AH_polytope(Z_o)\n",
    "    parametric_circumbody=circumbody.copy()\n",
    "    parametric_circumbody.P.h=circumbody.P.h*beta\n",
    "    Theta,*_=pp.subset(program,Z_i,parametric_circumbody,k=-1)\n",
    "    program.AddLinearCost(np.eye(1),np.zeros((1)),beta)\n",
    "    result=gurobi_solver.Solve(program,None,None)\n",
    "    if result.is_success():\n",
    "        alpha=1/result.GetSolution(beta)[0]\n",
    "        return alpha\n",
    "    else:\n",
    "        print(\"optimization failed\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 31,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "==================== \n",
      " n= 3\n",
      "3 5 0\n",
      "3 5 1\n",
      "3 5 2\n",
      "3 5 3\n",
      "3 5 4\n",
      "3 5 5\n",
      "3 5 6\n",
      "3 5 7\n",
      "3 5 8\n",
      "3 5 9\n",
      "3 5 10\n",
      "3 5 11\n",
      "3 5 12\n",
      "3 5 13\n",
      "3 5 14\n",
      "3 5 15\n",
      "3 5 16\n",
      "3 5 17\n",
      "3 5 18\n",
      "3 5 19\n",
      "3 5 20\n",
      "3 5 21\n",
      "3 5 22\n",
      "3 5 23\n",
      "3 5 24\n",
      "3 5 25\n",
      "3 5 26\n",
      "3 5 27\n",
      "3 5 28\n",
      "3 5 29\n",
      "3 5 30\n",
      "3 5 31\n",
      "3 5 32\n",
      "3 5 33\n",
      "3 5 34\n",
      "3 5 35\n",
      "3 5 36\n",
      "3 5 37\n",
      "3 5 38\n",
      "3 5 39\n",
      "3 5 40\n",
      "3 5 41\n",
      "3 5 42\n",
      "3 5 43\n",
      "3 5 44\n",
      "3 5 45\n",
      "3 5 46\n",
      "3 5 47\n",
      "3 5 48\n",
      "3 5 49\n",
      "3 5 50\n",
      "3 5 51\n",
      "3 5 52\n",
      "3 5 53\n",
      "3 5 54\n",
      "3 5 55\n",
      "3 5 56\n",
      "3 5 57\n",
      "3 5 58\n",
      "3 5 59\n",
      "3 5 60\n",
      "3 5 61\n",
      "3 5 62\n",
      "3 5 63\n",
      "3 5 64\n",
      "3 5 65\n",
      "3 5 66\n",
      "3 5 67\n",
      "3 5 68\n",
      "3 5 69\n",
      "3 5 70\n",
      "3 5 71\n",
      "3 5 72\n",
      "3 5 73\n",
      "3 5 74\n",
      "3 5 75\n",
      "3 5 76\n",
      "3 5 77\n",
      "3 5 78\n",
      "3 5 79\n",
      "3 5 80\n",
      "3 5 81\n",
      "3 5 82\n",
      "3 5 83\n",
      "3 5 84\n",
      "3 5 85\n",
      "3 5 86\n",
      "3 5 87\n",
      "3 5 88\n",
      "3 5 89\n",
      "3 5 90\n",
      "3 5 91\n",
      "3 5 92\n",
      "3 5 93\n",
      "3 5 94\n",
      "3 5 95\n",
      "3 5 96\n",
      "3 5 97\n",
      "3 5 98\n",
      "3 5 99\n",
      "3 5 100\n",
      "3 5 101\n",
      "3 5 102\n",
      "3 5 103\n",
      "3 5 104\n",
      "3 5 105\n",
      "3 5 106\n",
      "3 5 107\n",
      "3 5 108\n",
      "3 5 109\n",
      "3 5 110\n",
      "3 5 111\n",
      "3 5 112\n",
      "3 5 113\n",
      "3 5 114\n",
      "3 5 115\n",
      "3 5 116\n",
      "3 5 117\n",
      "3 5 118\n",
      "3 5 119\n",
      "3 5 120\n",
      "3 5 121\n",
      "3 5 122\n",
      "3 5 123\n",
      "3 5 124\n",
      "3 5 125\n",
      "3 5 126\n",
      "3 5 127\n",
      "3 5 128\n",
      "3 5 129\n",
      "3 5 130\n",
      "3 5 131\n",
      "3 5 132\n",
      "3 5 133\n",
      "3 5 134\n",
      "3 5 135\n",
      "3 5 136\n",
      "3 5 137\n",
      "3 5 138\n",
      "3 5 139\n",
      "3 5 140\n",
      "3 5 141\n",
      "3 5 142\n",
      "3 5 143\n",
      "3 5 144\n",
      "3 5 145\n",
      "3 5 146\n",
      "3 5 147\n",
      "3 5 148\n",
      "3 5 149\n",
      "3 5 150\n",
      "3 5 151\n",
      "3 5 152\n",
      "3 5 153\n",
      "3 5 154\n",
      "3 5 155\n",
      "3 5 156\n",
      "3 5 157\n",
      "3 5 158\n",
      "3 5 159\n",
      "3 5 160\n",
      "3 5 161\n",
      "3 5 162\n",
      "3 5 163\n",
      "3 5 164\n",
      "3 5 165\n",
      "3 5 166\n",
      "3 5 167\n",
      "3 5 168\n",
      "3 5 169\n",
      "3 5 170\n",
      "3 5 171\n",
      "3 5 172\n",
      "3 5 173\n",
      "3 5 174\n",
      "3 5 175\n",
      "3 5 176\n",
      "3 5 177\n",
      "3 5 178\n",
      "3 5 179\n",
      "3 5 180\n",
      "3 5 181\n",
      "3 5 182\n",
      "3 5 183\n",
      "3 5 184\n",
      "3 5 185\n",
      "3 5 186\n",
      "3 5 187\n",
      "3 5 188\n",
      "3 5 189\n",
      "3 5 190\n",
      "3 5 191\n",
      "3 5 192\n",
      "3 5 193\n",
      "3 5 194\n",
      "3 5 195\n",
      "3 5 196\n",
      "3 5 197\n",
      "3 5 198\n",
      "3 5 199\n",
      "3 5 200\n",
      "3 5 201\n",
      "3 5 202\n",
      "3 5 203\n",
      "3 5 204\n",
      "3 5 205\n",
      "3 5 206\n",
      "3 5 207\n",
      "3 5 208\n",
      "3 5 209\n",
      "3 5 210\n",
      "3 5 211\n",
      "3 5 212\n",
      "3 5 213\n",
      "3 5 214\n",
      "3 5 215\n",
      "3 5 216\n",
      "3 5 217\n",
      "3 5 218\n",
      "3 5 219\n",
      "3 5 220\n",
      "3 5 221\n",
      "3 5 222\n",
      "3 5 223\n",
      "3 5 224\n",
      "3 5 225\n",
      "3 5 226\n",
      "3 5 227\n",
      "3 5 228\n",
      "3 5 229\n",
      "3 5 230\n",
      "3 5 231\n",
      "3 5 232\n",
      "3 5 233\n",
      "3 5 234\n",
      "3 5 235\n",
      "3 5 236\n",
      "3 5 237\n",
      "3 5 238\n",
      "3 5 239\n",
      "3 5 240\n",
      "3 5 241\n",
      "3 5 242\n",
      "3 5 243\n",
      "3 5 244\n",
      "3 5 245\n",
      "3 5 246\n",
      "3 5 247\n",
      "3 5 248\n",
      "3 5 249\n",
      "3 5 250\n",
      "3 5 251\n",
      "3 5 252\n",
      "3 5 253\n",
      "3 5 254\n",
      "3 5 255\n",
      "3 5 256\n",
      "3 5 257\n",
      "3 5 258\n",
      "3 5 259\n",
      "3 5 260\n",
      "3 5 261\n",
      "3 5 262\n",
      "3 5 263\n",
      "3 5 264\n",
      "3 5 265\n",
      "3 5 266\n",
      "3 5 267\n",
      "3 5 268\n",
      "3 5 269\n",
      "3 5 270\n",
      "3 5 271\n",
      "3 5 272\n",
      "3 5 273\n",
      "3 5 274\n",
      "3 5 275\n",
      "3 5 276\n",
      "3 5 277\n",
      "3 5 278\n",
      "3 5 279\n",
      "3 5 280\n",
      "3 5 281\n",
      "3 5 282\n",
      "3 5 283\n",
      "3 5 284\n",
      "3 5 285\n",
      "3 5 286\n",
      "3 5 287\n",
      "3 5 288\n",
      "3 5 289\n",
      "3 5 290\n",
      "3 5 291\n",
      "3 5 292\n",
      "3 5 293\n",
      "3 5 294\n",
      "3 5 295\n",
      "3 5 296\n",
      "3 5 297\n",
      "3 5 298\n",
      "3 5 299\n",
      "3 5 300\n",
      "3 5 301\n",
      "3 5 302\n",
      "3 5 303\n",
      "3 5 304\n",
      "3 5 305\n",
      "3 5 306\n",
      "3 5 307\n",
      "3 5 308\n",
      "3 5 309\n",
      "3 5 310\n",
      "3 5 311\n",
      "3 5 312\n",
      "3 5 313\n",
      "3 5 314\n",
      "3 5 315\n",
      "3 5 316\n",
      "3 5 317\n",
      "3 5 318\n",
      "3 5 319\n",
      "3 5 320\n",
      "3 5 321\n",
      "3 5 322\n",
      "3 5 323\n",
      "3 5 324\n",
      "3 5 325\n",
      "3 5 326\n",
      "3 5 327\n",
      "3 5 328\n",
      "3 5 329\n",
      "3 5 330\n",
      "3 5 331\n",
      "3 5 332\n",
      "3 5 333\n",
      "3 5 334\n",
      "3 5 335\n",
      "3 5 336\n",
      "3 5 337\n",
      "3 5 338\n",
      "3 5 339\n",
      "3 5 340\n",
      "3 5 341\n",
      "3 5 342\n",
      "3 5 343\n",
      "3 5 344\n",
      "3 5 345\n",
      "3 5 346\n",
      "3 5 347\n",
      "3 5 348\n",
      "3 5 349\n",
      "3 5 350\n",
      "3 5 351\n",
      "3 5 352\n",
      "3 5 353\n",
      "3 5 354\n",
      "3 5 355\n",
      "3 5 356\n",
      "3 5 357\n",
      "3 5 358\n",
      "3 5 359\n",
      "3 5 360\n",
      "3 5 361\n",
      "3 5 362\n",
      "3 5 363\n",
      "3 5 364\n",
      "3 5 365\n",
      "3 5 366\n",
      "3 5 367\n",
      "3 5 368\n",
      "3 5 369\n",
      "3 5 370\n",
      "3 5 371\n",
      "3 5 372\n",
      "3 5 373\n",
      "3 5 374\n",
      "3 5 375\n",
      "3 5 376\n",
      "3 5 377\n",
      "3 5 378\n",
      "3 5 379\n",
      "3 5 380\n",
      "3 5 381\n",
      "3 5 382\n",
      "3 5 383\n",
      "3 5 384\n",
      "3 5 385\n",
      "3 5 386\n",
      "3 5 387\n",
      "3 5 388\n",
      "3 5 389\n",
      "3 5 390\n",
      "3 5 391\n",
      "3 5 392\n",
      "3 5 393\n",
      "3 5 394\n",
      "3 5 395\n",
      "3 5 396\n",
      "3 5 397\n",
      "3 5 398\n",
      "3 5 399\n",
      "3 5 400\n",
      "3 5 401\n",
      "3 5 402\n",
      "3 5 403\n",
      "3 5 404\n",
      "3 5 405\n",
      "3 5 406\n",
      "3 5 407\n",
      "3 5 408\n",
      "3 5 409\n",
      "3 5 410\n",
      "3 5 411\n",
      "3 5 412\n",
      "3 5 413\n",
      "3 5 414\n",
      "3 5 415\n",
      "3 5 416\n",
      "3 5 417\n",
      "3 5 418\n",
      "3 5 419\n",
      "3 5 420\n",
      "3 5 421\n",
      "3 5 422\n",
      "3 5 423\n",
      "3 5 424\n",
      "3 5 425\n",
      "3 5 426\n",
      "3 5 427\n",
      "3 5 428\n",
      "3 5 429\n",
      "3 5 430\n",
      "3 5 431\n",
      "3 5 432\n",
      "3 5 433\n",
      "3 5 434\n",
      "3 5 435\n",
      "3 5 436\n",
      "3 5 437\n",
      "3 5 438\n",
      "3 5 439\n",
      "3 5 440\n",
      "3 5 441\n",
      "3 5 442\n",
      "3 5 443\n",
      "3 5 444\n",
      "3 5 445\n",
      "3 5 446\n",
      "3 5 447\n",
      "3 5 448\n",
      "3 5 449\n",
      "3 5 450\n",
      "3 5 451\n",
      "3 5 452\n",
      "3 5 453\n",
      "3 5 454\n",
      "3 5 455\n",
      "3 5 456\n",
      "3 5 457\n",
      "3 5 458\n",
      "3 5 459\n",
      "3 5 460\n",
      "3 5 461\n",
      "3 5 462\n",
      "3 5 463\n",
      "3 5 464\n",
      "3 5 465\n",
      "3 5 466\n",
      "3 5 467\n",
      "3 5 468\n",
      "3 5 469\n",
      "3 5 470\n",
      "3 5 471\n",
      "3 5 472\n",
      "3 5 473\n",
      "3 5 474\n",
      "3 5 475\n",
      "3 5 476\n",
      "3 5 477\n",
      "3 5 478\n",
      "3 5 479\n",
      "3 5 480\n",
      "3 5 481\n",
      "3 5 482\n",
      "3 5 483\n",
      "3 5 484\n",
      "3 5 485\n",
      "3 5 486\n",
      "3 5 487\n",
      "3 5 488\n",
      "3 5 489\n",
      "3 5 490\n",
      "3 5 491\n",
      "3 5 492\n",
      "3 5 493\n",
      "3 5 494\n",
      "3 5 495\n",
      "3 5 496\n",
      "3 5 497\n",
      "3 5 498\n",
      "3 5 499\n",
      "3 10 0\n",
      "3 10 1\n",
      "3 10 2\n",
      "3 10 3\n",
      "3 10 4\n",
      "3 10 5\n",
      "3 10 6\n",
      "3 10 7\n",
      "3 10 8\n",
      "3 10 9\n",
      "3 10 10\n",
      "3 10 11\n",
      "3 10 12\n",
      "3 10 13\n",
      "3 10 14\n",
      "3 10 15\n",
      "3 10 16\n",
      "3 10 17\n",
      "3 10 18\n",
      "3 10 19\n",
      "3 10 20\n",
      "3 10 21\n",
      "3 10 22\n",
      "3 10 23\n",
      "3 10 24\n",
      "3 10 25\n",
      "3 10 26\n",
      "3 10 27\n",
      "3 10 28\n",
      "3 10 29\n",
      "3 10 30\n",
      "3 10 31\n",
      "3 10 32\n",
      "3 10 33\n",
      "3 10 34\n",
      "3 10 35\n",
      "3 10 36\n",
      "3 10 37\n",
      "3 10 38\n",
      "3 10 39\n",
      "3 10 40\n",
      "3 10 41\n",
      "3 10 42\n",
      "3 10 43\n",
      "3 10 44\n",
      "3 10 45\n",
      "3 10 46\n",
      "3 10 47\n",
      "3 10 48\n",
      "3 10 49\n",
      "3 10 50\n",
      "3 10 51\n",
      "3 10 52\n",
      "3 10 53\n",
      "3 10 54\n",
      "3 10 55\n",
      "3 10 56\n",
      "3 10 57\n",
      "3 10 58\n",
      "3 10 59\n",
      "3 10 60\n",
      "3 10 61\n",
      "3 10 62\n",
      "3 10 63\n",
      "3 10 64\n",
      "3 10 65\n",
      "3 10 66\n",
      "3 10 67\n",
      "3 10 68\n",
      "3 10 69\n",
      "3 10 70\n",
      "3 10 71\n",
      "3 10 72\n",
      "3 10 73\n",
      "3 10 74\n",
      "3 10 75\n",
      "3 10 76\n",
      "3 10 77\n",
      "3 10 78\n",
      "3 10 79\n",
      "3 10 80\n",
      "3 10 81\n",
      "3 10 82\n",
      "3 10 83\n",
      "3 10 84\n",
      "3 10 85\n",
      "3 10 86\n",
      "3 10 87\n",
      "3 10 88\n",
      "3 10 89\n",
      "3 10 90\n",
      "3 10 91\n",
      "3 10 92\n",
      "3 10 93\n",
      "3 10 94\n",
      "3 10 95\n",
      "3 10 96\n",
      "3 10 97\n",
      "3 10 98\n",
      "3 10 99\n",
      "3 10 100\n",
      "3 10 101\n",
      "3 10 102\n",
      "3 10 103\n",
      "3 10 104\n",
      "3 10 105\n",
      "3 10 106\n",
      "3 10 107\n",
      "3 10 108\n",
      "3 10 109\n",
      "3 10 110\n",
      "3 10 111\n",
      "3 10 112\n",
      "3 10 113\n",
      "3 10 114\n",
      "3 10 115\n",
      "3 10 116\n",
      "3 10 117\n",
      "3 10 118\n",
      "3 10 119\n",
      "3 10 120\n",
      "3 10 121\n",
      "3 10 122\n",
      "3 10 123\n",
      "3 10 124\n",
      "3 10 125\n",
      "3 10 126\n",
      "3 10 127\n",
      "3 10 128\n",
      "3 10 129\n",
      "3 10 130\n",
      "3 10 131\n",
      "3 10 132\n",
      "3 10 133\n",
      "3 10 134\n",
      "3 10 135\n",
      "3 10 136\n",
      "3 10 137\n",
      "3 10 138\n",
      "3 10 139\n",
      "3 10 140\n",
      "3 10 141\n",
      "3 10 142\n",
      "3 10 143\n",
      "3 10 144\n",
      "3 10 145\n",
      "3 10 146\n",
      "3 10 147\n",
      "3 10 148\n",
      "3 10 149\n",
      "3 10 150\n",
      "3 10 151\n",
      "3 10 152\n",
      "3 10 153\n",
      "3 10 154\n",
      "3 10 155\n",
      "3 10 156\n",
      "3 10 157\n",
      "3 10 158\n",
      "3 10 159\n",
      "3 10 160\n",
      "3 10 161\n",
      "3 10 162\n",
      "3 10 163\n",
      "3 10 164\n",
      "3 10 165\n",
      "3 10 166\n",
      "3 10 167\n",
      "3 10 168\n",
      "3 10 169\n",
      "3 10 170\n",
      "3 10 171\n",
      "3 10 172\n",
      "3 10 173\n",
      "3 10 174\n",
      "3 10 175\n",
      "3 10 176\n",
      "3 10 177\n",
      "3 10 178\n",
      "3 10 179\n",
      "3 10 180\n",
      "3 10 181\n",
      "3 10 182\n",
      "3 10 183\n",
      "3 10 184\n",
      "3 10 185\n",
      "3 10 186\n",
      "3 10 187\n",
      "3 10 188\n",
      "3 10 189\n",
      "3 10 190\n",
      "3 10 191\n",
      "3 10 192\n",
      "3 10 193\n",
      "3 10 194\n",
      "3 10 195\n",
      "3 10 196\n",
      "3 10 197\n",
      "3 10 198\n",
      "3 10 199\n",
      "3 10 200\n",
      "3 10 201\n",
      "3 10 202\n",
      "3 10 203\n",
      "3 10 204\n",
      "3 10 205\n",
      "3 10 206\n",
      "3 10 207\n",
      "3 10 208\n",
      "3 10 209\n",
      "3 10 210\n",
      "3 10 211\n",
      "3 10 212\n",
      "3 10 213\n",
      "3 10 214\n",
      "3 10 215\n",
      "3 10 216\n",
      "3 10 217\n",
      "3 10 218\n",
      "3 10 219\n",
      "3 10 220\n",
      "3 10 221\n",
      "3 10 222\n",
      "3 10 223\n",
      "3 10 224\n",
      "3 10 225\n",
      "3 10 226\n",
      "3 10 227\n",
      "3 10 228\n",
      "3 10 229\n",
      "3 10 230\n",
      "3 10 231\n",
      "3 10 232\n",
      "3 10 233\n",
      "3 10 234\n",
      "3 10 235\n",
      "3 10 236\n",
      "3 10 237\n",
      "3 10 238\n",
      "3 10 239\n",
      "3 10 240\n",
      "3 10 241\n",
      "3 10 242\n",
      "3 10 243\n",
      "3 10 244\n",
      "3 10 245\n",
      "3 10 246\n",
      "3 10 247\n",
      "3 10 248\n",
      "3 10 249\n",
      "3 10 250\n",
      "3 10 251\n",
      "3 10 252\n",
      "3 10 253\n",
      "3 10 254\n",
      "3 10 255\n",
      "3 10 256\n",
      "3 10 257\n",
      "3 10 258\n",
      "3 10 259\n",
      "3 10 260\n",
      "3 10 261\n",
      "3 10 262\n",
      "3 10 263\n",
      "3 10 264\n",
      "3 10 265\n",
      "3 10 266\n",
      "3 10 267\n",
      "3 10 268\n",
      "3 10 269\n",
      "3 10 270\n",
      "3 10 271\n",
      "3 10 272\n",
      "3 10 273\n",
      "3 10 274\n",
      "3 10 275\n",
      "3 10 276\n",
      "3 10 277\n",
      "3 10 278\n",
      "3 10 279\n",
      "3 10 280\n",
      "3 10 281\n",
      "3 10 282\n",
      "3 10 283\n",
      "3 10 284\n",
      "3 10 285\n",
      "3 10 286\n",
      "3 10 287\n",
      "3 10 288\n",
      "3 10 289\n",
      "3 10 290\n",
      "3 10 291\n",
      "3 10 292\n",
      "3 10 293\n",
      "3 10 294\n",
      "3 10 295\n",
      "3 10 296\n",
      "3 10 297\n",
      "3 10 298\n",
      "3 10 299\n",
      "3 10 300\n",
      "3 10 301\n",
      "3 10 302\n",
      "3 10 303\n",
      "3 10 304\n",
      "3 10 305\n",
      "3 10 306\n",
      "3 10 307\n",
      "3 10 308\n",
      "3 10 309\n",
      "3 10 310\n",
      "3 10 311\n",
      "3 10 312\n",
      "3 10 313\n",
      "3 10 314\n",
      "3 10 315\n",
      "3 10 316\n",
      "3 10 317\n",
      "3 10 318\n",
      "3 10 319\n",
      "3 10 320\n",
      "3 10 321\n",
      "3 10 322\n",
      "3 10 323\n",
      "3 10 324\n",
      "3 10 325\n",
      "3 10 326\n",
      "3 10 327\n",
      "3 10 328\n",
      "3 10 329\n",
      "3 10 330\n",
      "3 10 331\n",
      "3 10 332\n",
      "3 10 333\n",
      "3 10 334\n",
      "3 10 335\n",
      "3 10 336\n",
      "3 10 337\n",
      "3 10 338\n",
      "3 10 339\n",
      "3 10 340\n",
      "3 10 341\n",
      "3 10 342\n",
      "3 10 343\n",
      "3 10 344\n",
      "3 10 345\n",
      "3 10 346\n",
      "3 10 347\n",
      "3 10 348\n",
      "3 10 349\n",
      "3 10 350\n",
      "3 10 351\n",
      "3 10 352\n",
      "3 10 353\n",
      "3 10 354\n",
      "3 10 355\n",
      "3 10 356\n",
      "3 10 357\n",
      "3 10 358\n",
      "3 10 359\n",
      "3 10 360\n",
      "3 10 361\n",
      "3 10 362\n",
      "3 10 363\n",
      "3 10 364\n",
      "3 10 365\n",
      "3 10 366\n",
      "3 10 367\n",
      "3 10 368\n",
      "3 10 369\n",
      "3 10 370\n",
      "3 10 371\n",
      "3 10 372\n",
      "3 10 373\n",
      "3 10 374\n",
      "3 10 375\n",
      "3 10 376\n",
      "3 10 377\n",
      "3 10 378\n",
      "3 10 379\n",
      "3 10 380\n",
      "3 10 381\n",
      "3 10 382\n",
      "3 10 383\n",
      "3 10 384\n",
      "3 10 385\n",
      "3 10 386\n",
      "3 10 387\n",
      "3 10 388\n",
      "3 10 389\n",
      "3 10 390\n",
      "3 10 391\n",
      "3 10 392\n",
      "3 10 393\n",
      "3 10 394\n",
      "3 10 395\n",
      "3 10 396\n",
      "3 10 397\n",
      "3 10 398\n",
      "3 10 399\n",
      "3 10 400\n",
      "3 10 401\n",
      "3 10 402\n",
      "3 10 403\n",
      "3 10 404\n",
      "3 10 405\n",
      "3 10 406\n",
      "3 10 407\n",
      "3 10 408\n",
      "3 10 409\n",
      "3 10 410\n",
      "3 10 411\n",
      "3 10 412\n",
      "3 10 413\n",
      "3 10 414\n",
      "3 10 415\n",
      "3 10 416\n",
      "3 10 417\n",
      "3 10 418\n",
      "3 10 419\n",
      "3 10 420\n",
      "3 10 421\n",
      "3 10 422\n",
      "3 10 423\n",
      "3 10 424\n",
      "3 10 425\n",
      "3 10 426\n",
      "3 10 427\n",
      "3 10 428\n",
      "3 10 429\n",
      "3 10 430\n",
      "3 10 431\n",
      "3 10 432\n",
      "3 10 433\n",
      "3 10 434\n",
      "3 10 435\n",
      "3 10 436\n",
      "3 10 437\n",
      "3 10 438\n",
      "3 10 439\n",
      "3 10 440\n",
      "3 10 441\n",
      "3 10 442\n",
      "3 10 443\n",
      "3 10 444\n",
      "3 10 445\n",
      "3 10 446\n",
      "3 10 447\n",
      "3 10 448\n",
      "3 10 449\n",
      "3 10 450\n",
      "3 10 451\n",
      "3 10 452\n",
      "3 10 453\n",
      "3 10 454\n",
      "3 10 455\n",
      "3 10 456\n",
      "3 10 457\n",
      "3 10 458\n",
      "3 10 459\n",
      "3 10 460\n",
      "3 10 461\n",
      "3 10 462\n",
      "3 10 463\n",
      "3 10 464\n",
      "3 10 465\n",
      "3 10 466\n",
      "3 10 467\n",
      "3 10 468\n",
      "3 10 469\n",
      "3 10 470\n",
      "3 10 471\n",
      "3 10 472\n",
      "3 10 473\n",
      "3 10 474\n",
      "3 10 475\n",
      "3 10 476\n",
      "3 10 477\n",
      "3 10 478\n",
      "3 10 479\n",
      "3 10 480\n",
      "3 10 481\n",
      "3 10 482\n",
      "3 10 483\n",
      "3 10 484\n",
      "3 10 485\n",
      "3 10 486\n",
      "3 10 487\n",
      "3 10 488\n",
      "3 10 489\n",
      "3 10 490\n",
      "3 10 491\n",
      "3 10 492\n",
      "3 10 493\n",
      "3 10 494\n",
      "3 10 495\n",
      "3 10 496\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "3 10 497\n",
      "3 10 498\n",
      "3 10 499\n",
      "3 15 0\n",
      "3 15 1\n",
      "3 15 2\n",
      "3 15 3\n",
      "3 15 4\n",
      "3 15 5\n",
      "3 15 6\n",
      "3 15 7\n",
      "3 15 8\n",
      "3 15 9\n",
      "3 15 10\n",
      "3 15 11\n",
      "3 15 12\n",
      "3 15 13\n",
      "3 15 14\n",
      "3 15 15\n",
      "3 15 16\n",
      "3 15 17\n",
      "3 15 18\n",
      "3 15 19\n",
      "3 15 20\n",
      "3 15 21\n",
      "3 15 22\n",
      "3 15 23\n",
      "3 15 24\n",
      "3 15 25\n",
      "3 15 26\n",
      "3 15 27\n",
      "3 15 28\n",
      "3 15 29\n",
      "3 15 30\n",
      "3 15 31\n",
      "3 15 32\n",
      "3 15 33\n",
      "3 15 34\n",
      "3 15 35\n",
      "3 15 36\n",
      "3 15 37\n",
      "3 15 38\n",
      "3 15 39\n",
      "3 15 40\n",
      "3 15 41\n",
      "3 15 42\n",
      "3 15 43\n",
      "3 15 44\n",
      "3 15 45\n",
      "3 15 46\n",
      "3 15 47\n",
      "3 15 48\n",
      "3 15 49\n",
      "3 15 50\n",
      "3 15 51\n",
      "3 15 52\n",
      "3 15 53\n",
      "3 15 54\n",
      "3 15 55\n",
      "3 15 56\n",
      "3 15 57\n",
      "3 15 58\n",
      "3 15 59\n",
      "3 15 60\n",
      "3 15 61\n",
      "3 15 62\n",
      "3 15 63\n",
      "3 15 64\n",
      "3 15 65\n",
      "3 15 66\n",
      "3 15 67\n",
      "3 15 68\n",
      "3 15 69\n",
      "3 15 70\n",
      "3 15 71\n",
      "3 15 72\n",
      "3 15 73\n",
      "3 15 74\n",
      "3 15 75\n",
      "3 15 76\n",
      "3 15 77\n",
      "3 15 78\n",
      "3 15 79\n",
      "3 15 80\n",
      "3 15 81\n",
      "3 15 82\n",
      "3 15 83\n",
      "3 15 84\n",
      "3 15 85\n",
      "3 15 86\n",
      "3 15 87\n",
      "3 15 88\n",
      "3 15 89\n",
      "3 15 90\n",
      "3 15 91\n",
      "3 15 92\n",
      "3 15 93\n",
      "3 15 94\n",
      "3 15 95\n",
      "3 15 96\n",
      "3 15 97\n",
      "3 15 98\n",
      "3 15 99\n",
      "3 15 100\n",
      "3 15 101\n",
      "3 15 102\n",
      "3 15 103\n",
      "3 15 104\n",
      "3 15 105\n",
      "3 15 106\n",
      "3 15 107\n",
      "3 15 108\n",
      "3 15 109\n",
      "3 15 110\n",
      "3 15 111\n",
      "3 15 112\n",
      "3 15 113\n",
      "3 15 114\n",
      "3 15 115\n",
      "3 15 116\n",
      "3 15 117\n",
      "3 15 118\n",
      "3 15 119\n",
      "3 15 120\n",
      "3 15 121\n",
      "3 15 122\n",
      "3 15 123\n",
      "3 15 124\n",
      "3 15 125\n",
      "3 15 126\n",
      "3 15 127\n",
      "3 15 128\n",
      "3 15 129\n",
      "3 15 130\n",
      "3 15 131\n",
      "3 15 132\n",
      "3 15 133\n",
      "3 15 134\n",
      "3 15 135\n",
      "3 15 136\n",
      "3 15 137\n",
      "3 15 138\n",
      "3 15 139\n",
      "3 15 140\n",
      "3 15 141\n",
      "3 15 142\n",
      "3 15 143\n",
      "3 15 144\n",
      "3 15 145\n",
      "3 15 146\n",
      "3 15 147\n",
      "3 15 148\n",
      "3 15 149\n",
      "3 15 150\n",
      "3 15 151\n",
      "3 15 152\n",
      "3 15 153\n",
      "3 15 154\n",
      "3 15 155\n",
      "3 15 156\n",
      "3 15 157\n",
      "3 15 158\n",
      "3 15 159\n",
      "3 15 160\n",
      "3 15 161\n",
      "3 15 162\n",
      "3 15 163\n",
      "3 15 164\n",
      "3 15 165\n",
      "3 15 166\n",
      "3 15 167\n",
      "3 15 168\n",
      "3 15 169\n",
      "3 15 170\n",
      "3 15 171\n",
      "3 15 172\n",
      "3 15 173\n",
      "3 15 174\n",
      "3 15 175\n",
      "3 15 176\n",
      "3 15 177\n",
      "3 15 178\n",
      "3 15 179\n",
      "3 15 180\n",
      "3 15 181\n",
      "3 15 182\n",
      "3 15 183\n",
      "3 15 184\n",
      "3 15 185\n",
      "3 15 186\n",
      "3 15 187\n",
      "3 15 188\n",
      "3 15 189\n",
      "3 15 190\n",
      "3 15 191\n",
      "3 15 192\n",
      "3 15 193\n",
      "3 15 194\n",
      "3 15 195\n",
      "3 15 196\n",
      "3 15 197\n",
      "3 15 198\n",
      "3 15 199\n",
      "3 15 200\n",
      "3 15 201\n",
      "3 15 202\n",
      "3 15 203\n",
      "3 15 204\n",
      "3 15 205\n",
      "3 15 206\n",
      "3 15 207\n",
      "3 15 208\n",
      "3 15 209\n",
      "3 15 210\n",
      "3 15 211\n",
      "3 15 212\n",
      "3 15 213\n",
      "3 15 214\n",
      "3 15 215\n",
      "3 15 216\n",
      "3 15 217\n",
      "3 15 218\n",
      "3 15 219\n",
      "3 15 220\n",
      "3 15 221\n",
      "3 15 222\n",
      "3 15 223\n",
      "3 15 224\n",
      "3 15 225\n",
      "3 15 226\n",
      "3 15 227\n",
      "3 15 228\n",
      "3 15 229\n",
      "3 15 230\n",
      "3 15 231\n",
      "3 15 232\n",
      "3 15 233\n",
      "3 15 234\n",
      "3 15 235\n",
      "3 15 236\n",
      "3 15 237\n",
      "3 15 238\n",
      "3 15 239\n",
      "3 15 240\n",
      "3 15 241\n",
      "3 15 242\n",
      "3 15 243\n",
      "3 15 244\n",
      "3 15 245\n",
      "3 15 246\n",
      "3 15 247\n",
      "3 15 248\n",
      "3 15 249\n",
      "3 15 250\n",
      "3 15 251\n",
      "3 15 252\n",
      "3 15 253\n",
      "3 15 254\n",
      "3 15 255\n",
      "3 15 256\n",
      "3 15 257\n",
      "3 15 258\n",
      "3 15 259\n",
      "3 15 260\n",
      "3 15 261\n",
      "3 15 262\n",
      "3 15 263\n",
      "3 15 264\n",
      "3 15 265\n",
      "3 15 266\n",
      "3 15 267\n",
      "3 15 268\n",
      "3 15 269\n",
      "3 15 270\n",
      "3 15 271\n",
      "3 15 272\n",
      "3 15 273\n",
      "3 15 274\n",
      "3 15 275\n",
      "3 15 276\n",
      "3 15 277\n",
      "3 15 278\n",
      "3 15 279\n",
      "3 15 280\n",
      "3 15 281\n",
      "3 15 282\n",
      "3 15 283\n",
      "3 15 284\n",
      "3 15 285\n",
      "3 15 286\n",
      "3 15 287\n",
      "3 15 288\n",
      "3 15 289\n",
      "3 15 290\n",
      "3 15 291\n",
      "3 15 292\n",
      "3 15 293\n",
      "3 15 294\n",
      "3 15 295\n",
      "3 15 296\n",
      "3 15 297\n",
      "3 15 298\n",
      "3 15 299\n",
      "3 15 300\n",
      "3 15 301\n",
      "3 15 302\n",
      "3 15 303\n",
      "3 15 304\n",
      "3 15 305\n",
      "3 15 306\n",
      "3 15 307\n",
      "3 15 308\n",
      "3 15 309\n",
      "3 15 310\n",
      "3 15 311\n",
      "3 15 312\n",
      "3 15 313\n",
      "3 15 314\n",
      "3 15 315\n",
      "3 15 316\n",
      "3 15 317\n",
      "3 15 318\n",
      "3 15 319\n",
      "3 15 320\n",
      "3 15 321\n",
      "3 15 322\n",
      "3 15 323\n",
      "3 15 324\n",
      "3 15 325\n",
      "3 15 326\n",
      "3 15 327\n",
      "3 15 328\n",
      "3 15 329\n",
      "3 15 330\n",
      "3 15 331\n",
      "3 15 332\n",
      "3 15 333\n",
      "3 15 334\n",
      "3 15 335\n",
      "3 15 336\n",
      "3 15 337\n",
      "3 15 338\n",
      "3 15 339\n",
      "3 15 340\n",
      "3 15 341\n",
      "3 15 342\n",
      "3 15 343\n",
      "3 15 344\n",
      "3 15 345\n",
      "3 15 346\n",
      "3 15 347\n",
      "3 15 348\n",
      "3 15 349\n",
      "3 15 350\n",
      "3 15 351\n",
      "3 15 352\n",
      "3 15 353\n",
      "3 15 354\n",
      "3 15 355\n",
      "3 15 356\n",
      "3 15 357\n",
      "3 15 358\n",
      "3 15 359\n",
      "3 15 360\n",
      "3 15 361\n",
      "3 15 362\n",
      "3 15 363\n",
      "3 15 364\n",
      "3 15 365\n",
      "3 15 366\n",
      "3 15 367\n",
      "3 15 368\n",
      "3 15 369\n",
      "3 15 370\n",
      "3 15 371\n",
      "3 15 372\n",
      "3 15 373\n",
      "3 15 374\n",
      "3 15 375\n",
      "3 15 376\n",
      "3 15 377\n",
      "3 15 378\n",
      "3 15 379\n",
      "3 15 380\n",
      "3 15 381\n",
      "3 15 382\n",
      "3 15 383\n",
      "3 15 384\n",
      "3 15 385\n",
      "3 15 386\n",
      "3 15 387\n",
      "3 15 388\n",
      "3 15 389\n",
      "3 15 390\n",
      "3 15 391\n",
      "3 15 392\n",
      "3 15 393\n",
      "3 15 394\n",
      "3 15 395\n",
      "3 15 396\n",
      "3 15 397\n",
      "3 15 398\n",
      "3 15 399\n",
      "3 15 400\n",
      "3 15 401\n",
      "3 15 402\n",
      "3 15 403\n",
      "3 15 404\n",
      "3 15 405\n",
      "3 15 406\n",
      "3 15 407\n",
      "3 15 408\n",
      "3 15 409\n",
      "3 15 410\n",
      "3 15 411\n",
      "3 15 412\n",
      "3 15 413\n",
      "3 15 414\n",
      "3 15 415\n",
      "3 15 416\n",
      "3 15 417\n",
      "3 15 418\n",
      "3 15 419\n",
      "3 15 420\n",
      "3 15 421\n",
      "3 15 422\n",
      "3 15 423\n",
      "3 15 424\n",
      "3 15 425\n",
      "3 15 426\n",
      "3 15 427\n",
      "3 15 428\n",
      "3 15 429\n",
      "3 15 430\n",
      "3 15 431\n",
      "3 15 432\n",
      "3 15 433\n",
      "3 15 434\n",
      "3 15 435\n",
      "3 15 436\n",
      "3 15 437\n",
      "3 15 438\n",
      "3 15 439\n",
      "3 15 440\n",
      "3 15 441\n",
      "3 15 442\n",
      "3 15 443\n",
      "3 15 444\n",
      "3 15 445\n",
      "3 15 446\n",
      "3 15 447\n",
      "3 15 448\n",
      "3 15 449\n",
      "3 15 450\n",
      "3 15 451\n",
      "3 15 452\n",
      "3 15 453\n",
      "3 15 454\n",
      "3 15 455\n",
      "3 15 456\n",
      "3 15 457\n",
      "3 15 458\n",
      "3 15 459\n",
      "3 15 460\n",
      "3 15 461\n",
      "3 15 462\n",
      "3 15 463\n",
      "3 15 464\n",
      "3 15 465\n",
      "3 15 466\n",
      "3 15 467\n",
      "3 15 468\n",
      "3 15 469\n",
      "3 15 470\n",
      "3 15 471\n",
      "3 15 472\n",
      "3 15 473\n",
      "3 15 474\n",
      "3 15 475\n",
      "3 15 476\n",
      "3 15 477\n",
      "3 15 478\n",
      "3 15 479\n",
      "3 15 480\n",
      "3 15 481\n",
      "3 15 482\n",
      "3 15 483\n",
      "3 15 484\n",
      "3 15 485\n",
      "3 15 486\n",
      "3 15 487\n",
      "3 15 488\n",
      "3 15 489\n",
      "3 15 490\n",
      "3 15 491\n",
      "3 15 492\n",
      "3 15 493\n",
      "3 15 494\n",
      "3 15 495\n",
      "3 15 496\n",
      "3 15 497\n",
      "3 15 498\n",
      "3 15 499\n",
      "3 20 0\n",
      "3 20 1\n",
      "3 20 2\n",
      "3 20 3\n",
      "3 20 4\n",
      "3 20 5\n",
      "3 20 6\n",
      "3 20 7\n",
      "3 20 8\n",
      "3 20 9\n",
      "3 20 10\n",
      "3 20 11\n",
      "3 20 12\n",
      "3 20 13\n",
      "3 20 14\n",
      "3 20 15\n",
      "3 20 16\n",
      "3 20 17\n",
      "3 20 18\n",
      "3 20 19\n",
      "3 20 20\n",
      "3 20 21\n",
      "3 20 22\n",
      "3 20 23\n",
      "3 20 24\n",
      "3 20 25\n",
      "3 20 26\n",
      "3 20 27\n",
      "3 20 28\n",
      "3 20 29\n",
      "3 20 30\n",
      "3 20 31\n",
      "3 20 32\n",
      "3 20 33\n",
      "3 20 34\n",
      "3 20 35\n",
      "3 20 36\n",
      "3 20 37\n",
      "3 20 38\n",
      "3 20 39\n",
      "3 20 40\n",
      "3 20 41\n",
      "3 20 42\n",
      "3 20 43\n",
      "3 20 44\n",
      "3 20 45\n",
      "3 20 46\n",
      "3 20 47\n",
      "3 20 48\n",
      "3 20 49\n",
      "3 20 50\n",
      "3 20 51\n",
      "3 20 52\n",
      "3 20 53\n",
      "3 20 54\n",
      "3 20 55\n",
      "3 20 56\n",
      "3 20 57\n",
      "3 20 58\n",
      "3 20 59\n",
      "3 20 60\n",
      "3 20 61\n",
      "3 20 62\n",
      "3 20 63\n",
      "3 20 64\n",
      "3 20 65\n",
      "3 20 66\n",
      "3 20 67\n",
      "3 20 68\n",
      "3 20 69\n",
      "3 20 70\n",
      "3 20 71\n",
      "3 20 72\n",
      "3 20 73\n",
      "3 20 74\n",
      "3 20 75\n",
      "3 20 76\n",
      "3 20 77\n",
      "3 20 78\n",
      "3 20 79\n",
      "3 20 80\n",
      "3 20 81\n",
      "3 20 82\n",
      "3 20 83\n",
      "3 20 84\n",
      "3 20 85\n",
      "3 20 86\n",
      "3 20 87\n",
      "3 20 88\n",
      "3 20 89\n",
      "3 20 90\n",
      "3 20 91\n",
      "3 20 92\n",
      "3 20 93\n",
      "3 20 94\n",
      "3 20 95\n",
      "3 20 96\n",
      "3 20 97\n",
      "3 20 98\n",
      "3 20 99\n",
      "3 20 100\n",
      "3 20 101\n",
      "3 20 102\n",
      "3 20 103\n",
      "3 20 104\n",
      "3 20 105\n",
      "3 20 106\n",
      "3 20 107\n",
      "3 20 108\n",
      "3 20 109\n",
      "3 20 110\n",
      "3 20 111\n",
      "3 20 112\n",
      "3 20 113\n",
      "3 20 114\n",
      "3 20 115\n",
      "3 20 116\n",
      "3 20 117\n",
      "3 20 118\n",
      "3 20 119\n",
      "3 20 120\n",
      "3 20 121\n",
      "3 20 122\n",
      "3 20 123\n",
      "3 20 124\n",
      "3 20 125\n",
      "3 20 126\n",
      "3 20 127\n",
      "3 20 128\n",
      "3 20 129\n",
      "3 20 130\n",
      "3 20 131\n",
      "3 20 132\n",
      "3 20 133\n",
      "3 20 134\n",
      "3 20 135\n",
      "3 20 136\n",
      "3 20 137\n",
      "3 20 138\n",
      "3 20 139\n",
      "3 20 140\n",
      "3 20 141\n",
      "3 20 142\n",
      "3 20 143\n",
      "3 20 144\n",
      "3 20 145\n",
      "3 20 146\n",
      "3 20 147\n",
      "3 20 148\n",
      "3 20 149\n",
      "3 20 150\n",
      "3 20 151\n",
      "3 20 152\n",
      "3 20 153\n",
      "3 20 154\n",
      "3 20 155\n",
      "3 20 156\n",
      "3 20 157\n",
      "3 20 158\n",
      "3 20 159\n",
      "3 20 160\n",
      "3 20 161\n",
      "3 20 162\n",
      "3 20 163\n",
      "3 20 164\n",
      "3 20 165\n",
      "3 20 166\n",
      "3 20 167\n",
      "3 20 168\n",
      "3 20 169\n",
      "3 20 170\n",
      "3 20 171\n",
      "3 20 172\n",
      "3 20 173\n",
      "3 20 174\n",
      "3 20 175\n",
      "3 20 176\n",
      "3 20 177\n",
      "3 20 178\n",
      "3 20 179\n",
      "3 20 180\n",
      "3 20 181\n",
      "3 20 182\n",
      "3 20 183\n",
      "3 20 184\n",
      "3 20 185\n",
      "3 20 186\n",
      "3 20 187\n",
      "3 20 188\n",
      "3 20 189\n",
      "3 20 190\n",
      "3 20 191\n",
      "3 20 192\n",
      "3 20 193\n",
      "3 20 194\n",
      "3 20 195\n",
      "3 20 196\n",
      "3 20 197\n",
      "3 20 198\n",
      "3 20 199\n",
      "3 20 200\n",
      "3 20 201\n",
      "3 20 202\n",
      "3 20 203\n",
      "3 20 204\n",
      "3 20 205\n",
      "3 20 206\n",
      "3 20 207\n",
      "3 20 208\n",
      "3 20 209\n",
      "3 20 210\n",
      "3 20 211\n",
      "3 20 212\n",
      "3 20 213\n",
      "3 20 214\n",
      "3 20 215\n",
      "3 20 216\n",
      "3 20 217\n",
      "3 20 218\n",
      "3 20 219\n",
      "3 20 220\n",
      "3 20 221\n",
      "3 20 222\n",
      "3 20 223\n",
      "3 20 224\n",
      "3 20 225\n",
      "3 20 226\n",
      "3 20 227\n",
      "3 20 228\n",
      "3 20 229\n",
      "3 20 230\n",
      "3 20 231\n",
      "3 20 232\n",
      "3 20 233\n",
      "3 20 234\n",
      "3 20 235\n",
      "3 20 236\n",
      "3 20 237\n",
      "3 20 238\n",
      "3 20 239\n",
      "3 20 240\n",
      "3 20 241\n",
      "3 20 242\n",
      "3 20 243\n",
      "3 20 244\n",
      "3 20 245\n",
      "3 20 246\n",
      "3 20 247\n",
      "3 20 248\n",
      "3 20 249\n",
      "3 20 250\n",
      "3 20 251\n",
      "3 20 252\n",
      "3 20 253\n",
      "3 20 254\n",
      "3 20 255\n",
      "3 20 256\n",
      "3 20 257\n",
      "3 20 258\n",
      "3 20 259\n",
      "3 20 260\n",
      "3 20 261\n",
      "3 20 262\n",
      "3 20 263\n",
      "3 20 264\n",
      "3 20 265\n",
      "3 20 266\n",
      "3 20 267\n",
      "3 20 268\n",
      "3 20 269\n",
      "3 20 270\n",
      "3 20 271\n",
      "3 20 272\n",
      "3 20 273\n",
      "3 20 274\n",
      "3 20 275\n",
      "3 20 276\n",
      "3 20 277\n",
      "3 20 278\n",
      "3 20 279\n",
      "3 20 280\n",
      "3 20 281\n",
      "3 20 282\n",
      "3 20 283\n",
      "3 20 284\n",
      "3 20 285\n",
      "3 20 286\n",
      "3 20 287\n",
      "3 20 288\n",
      "3 20 289\n",
      "3 20 290\n",
      "3 20 291\n",
      "3 20 292\n",
      "3 20 293\n",
      "3 20 294\n",
      "3 20 295\n",
      "3 20 296\n",
      "3 20 297\n",
      "3 20 298\n",
      "3 20 299\n",
      "3 20 300\n",
      "3 20 301\n",
      "3 20 302\n",
      "3 20 303\n",
      "3 20 304\n",
      "3 20 305\n",
      "3 20 306\n",
      "3 20 307\n",
      "3 20 308\n",
      "3 20 309\n",
      "3 20 310\n",
      "3 20 311\n",
      "3 20 312\n",
      "3 20 313\n",
      "3 20 314\n",
      "3 20 315\n",
      "3 20 316\n",
      "3 20 317\n",
      "3 20 318\n",
      "3 20 319\n",
      "3 20 320\n",
      "3 20 321\n",
      "3 20 322\n",
      "3 20 323\n",
      "3 20 324\n",
      "3 20 325\n",
      "3 20 326\n",
      "3 20 327\n",
      "3 20 328\n",
      "3 20 329\n",
      "3 20 330\n",
      "3 20 331\n",
      "3 20 332\n",
      "3 20 333\n",
      "3 20 334\n",
      "3 20 335\n",
      "3 20 336\n",
      "3 20 337\n",
      "3 20 338\n",
      "3 20 339\n",
      "3 20 340\n",
      "3 20 341\n",
      "3 20 342\n",
      "3 20 343\n",
      "3 20 344\n",
      "3 20 345\n",
      "3 20 346\n",
      "3 20 347\n",
      "3 20 348\n",
      "3 20 349\n",
      "3 20 350\n",
      "3 20 351\n",
      "3 20 352\n",
      "3 20 353\n",
      "3 20 354\n",
      "3 20 355\n",
      "3 20 356\n",
      "3 20 357\n",
      "3 20 358\n",
      "3 20 359\n",
      "3 20 360\n",
      "3 20 361\n",
      "3 20 362\n",
      "3 20 363\n",
      "3 20 364\n",
      "3 20 365\n",
      "3 20 366\n",
      "3 20 367\n",
      "3 20 368\n",
      "3 20 369\n",
      "3 20 370\n",
      "3 20 371\n",
      "3 20 372\n",
      "3 20 373\n",
      "3 20 374\n",
      "3 20 375\n",
      "3 20 376\n",
      "3 20 377\n",
      "3 20 378\n",
      "3 20 379\n",
      "3 20 380\n",
      "3 20 381\n",
      "3 20 382\n",
      "3 20 383\n",
      "3 20 384\n",
      "3 20 385\n",
      "3 20 386\n",
      "3 20 387\n",
      "3 20 388\n",
      "3 20 389\n",
      "3 20 390\n",
      "3 20 391\n",
      "3 20 392\n",
      "3 20 393\n",
      "3 20 394\n",
      "3 20 395\n",
      "3 20 396\n",
      "3 20 397\n",
      "3 20 398\n",
      "3 20 399\n",
      "3 20 400\n",
      "3 20 401\n",
      "3 20 402\n",
      "3 20 403\n",
      "3 20 404\n",
      "3 20 405\n",
      "3 20 406\n",
      "3 20 407\n",
      "3 20 408\n",
      "3 20 409\n",
      "3 20 410\n",
      "3 20 411\n",
      "3 20 412\n",
      "3 20 413\n",
      "3 20 414\n",
      "3 20 415\n",
      "3 20 416\n",
      "3 20 417\n",
      "3 20 418\n",
      "3 20 419\n",
      "3 20 420\n",
      "3 20 421\n",
      "3 20 422\n",
      "3 20 423\n",
      "3 20 424\n",
      "3 20 425\n",
      "3 20 426\n",
      "3 20 427\n",
      "3 20 428\n",
      "3 20 429\n",
      "3 20 430\n",
      "3 20 431\n",
      "3 20 432\n",
      "3 20 433\n",
      "3 20 434\n",
      "3 20 435\n",
      "3 20 436\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "3 20 437\n",
      "3 20 438\n",
      "3 20 439\n",
      "3 20 440\n",
      "3 20 441\n",
      "3 20 442\n",
      "3 20 443\n",
      "3 20 444\n",
      "3 20 445\n",
      "3 20 446\n",
      "3 20 447\n",
      "3 20 448\n",
      "3 20 449\n",
      "3 20 450\n",
      "3 20 451\n",
      "3 20 452\n",
      "3 20 453\n",
      "3 20 454\n",
      "3 20 455\n",
      "3 20 456\n",
      "3 20 457\n",
      "3 20 458\n",
      "3 20 459\n",
      "3 20 460\n",
      "3 20 461\n",
      "3 20 462\n",
      "3 20 463\n",
      "3 20 464\n",
      "3 20 465\n",
      "3 20 466\n",
      "3 20 467\n",
      "3 20 468\n",
      "3 20 469\n",
      "3 20 470\n",
      "3 20 471\n",
      "3 20 472\n",
      "3 20 473\n",
      "3 20 474\n",
      "3 20 475\n",
      "3 20 476\n",
      "3 20 477\n",
      "3 20 478\n",
      "3 20 479\n",
      "3 20 480\n",
      "3 20 481\n",
      "3 20 482\n",
      "3 20 483\n",
      "3 20 484\n",
      "3 20 485\n",
      "3 20 486\n",
      "3 20 487\n",
      "3 20 488\n",
      "3 20 489\n",
      "3 20 490\n",
      "3 20 491\n",
      "3 20 492\n",
      "3 20 493\n",
      "3 20 494\n",
      "3 20 495\n",
      "3 20 496\n",
      "3 20 497\n",
      "3 20 498\n",
      "3 20 499\n",
      "==================== \n",
      " n= 5\n",
      "5 10 0\n",
      "5 10 1\n",
      "5 10 2\n",
      "5 10 3\n",
      "5 10 4\n",
      "5 10 5\n",
      "5 10 6\n",
      "5 10 7\n",
      "5 10 8\n",
      "5 10 9\n",
      "5 10 10\n",
      "5 10 11\n",
      "5 10 12\n",
      "5 10 13\n",
      "5 10 14\n",
      "5 10 15\n",
      "5 10 16\n",
      "5 10 17\n",
      "5 10 18\n",
      "5 10 19\n",
      "5 10 20\n",
      "5 10 21\n",
      "5 10 22\n",
      "5 10 23\n",
      "5 10 24\n",
      "5 10 25\n",
      "5 10 26\n",
      "5 10 27\n",
      "5 10 28\n",
      "5 10 29\n",
      "5 10 30\n",
      "5 10 31\n",
      "5 10 32\n",
      "5 10 33\n",
      "5 10 34\n",
      "5 10 35\n",
      "5 10 36\n",
      "5 10 37\n",
      "5 10 38\n",
      "5 10 39\n",
      "5 10 40\n",
      "5 10 41\n",
      "5 10 42\n",
      "5 10 43\n",
      "5 10 44\n",
      "5 10 45\n",
      "5 10 46\n",
      "5 10 47\n",
      "5 10 48\n",
      "5 10 49\n",
      "5 10 50\n",
      "5 10 51\n",
      "5 10 52\n",
      "5 10 53\n",
      "5 10 54\n",
      "5 10 55\n",
      "5 10 56\n",
      "5 10 57\n",
      "5 10 58\n",
      "5 10 59\n",
      "5 10 60\n",
      "5 10 61\n",
      "5 10 62\n",
      "5 10 63\n",
      "5 10 64\n",
      "5 10 65\n",
      "5 10 66\n",
      "5 10 67\n",
      "5 10 68\n",
      "5 10 69\n",
      "5 10 70\n",
      "5 10 71\n",
      "5 10 72\n",
      "5 10 73\n",
      "5 10 74\n",
      "5 10 75\n",
      "5 10 76\n",
      "5 10 77\n",
      "5 10 78\n",
      "5 10 79\n",
      "5 10 80\n",
      "5 10 81\n",
      "5 10 82\n",
      "5 10 83\n",
      "5 10 84\n",
      "5 10 85\n",
      "5 10 86\n",
      "5 10 87\n",
      "5 10 88\n",
      "5 10 89\n",
      "5 10 90\n",
      "5 10 91\n",
      "5 10 92\n",
      "5 10 93\n",
      "5 10 94\n",
      "5 10 95\n",
      "5 10 96\n",
      "5 10 97\n",
      "5 10 98\n",
      "5 10 99\n",
      "5 10 100\n",
      "5 10 101\n",
      "5 10 102\n",
      "5 10 103\n",
      "5 10 104\n",
      "5 10 105\n",
      "5 10 106\n",
      "5 10 107\n",
      "5 10 108\n",
      "5 10 109\n",
      "5 10 110\n",
      "5 10 111\n",
      "5 10 112\n",
      "5 10 113\n",
      "5 10 114\n",
      "5 10 115\n",
      "5 10 116\n",
      "5 10 117\n",
      "5 10 118\n",
      "5 10 119\n",
      "5 10 120\n",
      "5 10 121\n",
      "5 10 122\n",
      "5 10 123\n",
      "5 10 124\n",
      "5 10 125\n",
      "5 10 126\n",
      "5 10 127\n",
      "5 10 128\n",
      "5 10 129\n",
      "5 10 130\n",
      "5 10 131\n",
      "5 10 132\n",
      "5 10 133\n",
      "5 10 134\n",
      "5 10 135\n",
      "5 10 136\n",
      "5 10 137\n",
      "5 10 138\n",
      "5 10 139\n",
      "5 10 140\n",
      "5 10 141\n",
      "5 10 142\n",
      "5 10 143\n",
      "5 10 144\n",
      "5 10 145\n",
      "5 10 146\n",
      "5 10 147\n",
      "5 10 148\n",
      "5 10 149\n",
      "5 10 150\n",
      "5 10 151\n",
      "5 10 152\n",
      "5 10 153\n",
      "5 10 154\n",
      "5 10 155\n",
      "5 10 156\n",
      "5 10 157\n",
      "5 10 158\n",
      "5 10 159\n",
      "5 10 160\n",
      "5 10 161\n",
      "5 10 162\n",
      "5 10 163\n",
      "5 10 164\n",
      "5 10 165\n",
      "5 10 166\n",
      "5 10 167\n",
      "5 10 168\n",
      "5 10 169\n",
      "5 10 170\n",
      "5 10 171\n",
      "5 10 172\n",
      "5 10 173\n",
      "5 10 174\n",
      "5 10 175\n",
      "5 10 176\n",
      "5 10 177\n",
      "5 10 178\n",
      "5 10 179\n",
      "5 10 180\n",
      "5 10 181\n",
      "5 10 182\n",
      "5 10 183\n",
      "5 10 184\n",
      "5 10 185\n",
      "5 10 186\n",
      "5 10 187\n",
      "5 10 188\n",
      "5 10 189\n",
      "5 10 190\n",
      "5 10 191\n",
      "5 10 192\n",
      "5 10 193\n",
      "5 10 194\n",
      "5 10 195\n",
      "5 10 196\n",
      "5 10 197\n",
      "5 10 198\n",
      "5 10 199\n",
      "5 10 200\n",
      "5 10 201\n",
      "5 10 202\n",
      "5 10 203\n",
      "5 10 204\n",
      "5 10 205\n",
      "5 10 206\n",
      "5 10 207\n",
      "5 10 208\n",
      "5 10 209\n",
      "5 10 210\n",
      "5 10 211\n",
      "5 10 212\n",
      "5 10 213\n",
      "5 10 214\n",
      "5 10 215\n",
      "5 10 216\n",
      "5 10 217\n",
      "5 10 218\n",
      "5 10 219\n",
      "5 10 220\n",
      "5 10 221\n",
      "5 10 222\n",
      "5 10 223\n",
      "5 10 224\n",
      "5 10 225\n",
      "5 10 226\n",
      "5 10 227\n",
      "5 10 228\n",
      "5 10 229\n",
      "5 10 230\n",
      "5 10 231\n",
      "5 10 232\n",
      "5 10 233\n",
      "5 10 234\n",
      "5 10 235\n",
      "5 10 236\n",
      "5 10 237\n",
      "5 10 238\n",
      "5 10 239\n",
      "5 10 240\n",
      "5 10 241\n",
      "5 10 242\n",
      "5 10 243\n",
      "5 10 244\n",
      "5 10 245\n",
      "5 10 246\n",
      "5 10 247\n",
      "5 10 248\n",
      "5 10 249\n",
      "5 10 250\n",
      "5 10 251\n",
      "5 10 252\n",
      "5 10 253\n",
      "5 10 254\n",
      "5 10 255\n",
      "5 10 256\n",
      "5 10 257\n",
      "5 10 258\n",
      "5 10 259\n",
      "5 10 260\n",
      "5 10 261\n",
      "5 10 262\n",
      "5 10 263\n",
      "5 10 264\n",
      "5 10 265\n",
      "5 10 266\n",
      "5 10 267\n",
      "5 10 268\n",
      "5 10 269\n",
      "5 10 270\n",
      "5 10 271\n",
      "5 10 272\n",
      "5 10 273\n",
      "5 10 274\n",
      "5 10 275\n",
      "5 10 276\n",
      "5 10 277\n",
      "5 10 278\n",
      "5 10 279\n",
      "5 10 280\n",
      "5 10 281\n",
      "5 10 282\n",
      "5 10 283\n",
      "5 10 284\n",
      "5 10 285\n",
      "5 10 286\n",
      "5 10 287\n",
      "5 10 288\n",
      "5 10 289\n",
      "5 10 290\n",
      "5 10 291\n",
      "5 10 292\n",
      "5 10 293\n",
      "5 10 294\n",
      "5 10 295\n",
      "5 10 296\n",
      "5 10 297\n",
      "5 10 298\n",
      "5 10 299\n",
      "5 10 300\n",
      "5 10 301\n",
      "5 10 302\n",
      "5 10 303\n",
      "5 10 304\n",
      "5 10 305\n",
      "5 10 306\n",
      "5 10 307\n",
      "5 10 308\n",
      "5 10 309\n",
      "5 10 310\n",
      "5 10 311\n",
      "5 10 312\n",
      "5 10 313\n",
      "5 10 314\n",
      "5 10 315\n",
      "5 10 316\n",
      "5 10 317\n",
      "5 10 318\n",
      "5 10 319\n",
      "5 10 320\n",
      "5 10 321\n",
      "5 10 322\n",
      "5 10 323\n",
      "5 10 324\n",
      "5 10 325\n",
      "5 10 326\n",
      "5 10 327\n",
      "5 10 328\n",
      "5 10 329\n",
      "5 10 330\n",
      "5 10 331\n",
      "5 10 332\n",
      "5 10 333\n",
      "5 10 334\n",
      "5 10 335\n",
      "5 10 336\n",
      "5 10 337\n",
      "5 10 338\n",
      "5 10 339\n",
      "5 10 340\n",
      "5 10 341\n",
      "5 10 342\n",
      "5 10 343\n",
      "5 10 344\n",
      "5 10 345\n",
      "5 10 346\n",
      "5 10 347\n",
      "5 10 348\n",
      "5 10 349\n",
      "5 10 350\n",
      "5 10 351\n",
      "5 10 352\n",
      "5 10 353\n",
      "5 10 354\n",
      "5 10 355\n",
      "5 10 356\n",
      "5 10 357\n",
      "5 10 358\n",
      "5 10 359\n",
      "5 10 360\n",
      "5 10 361\n",
      "5 10 362\n",
      "5 10 363\n",
      "5 10 364\n",
      "5 10 365\n",
      "5 10 366\n",
      "5 10 367\n",
      "5 10 368\n",
      "5 10 369\n",
      "5 10 370\n",
      "5 10 371\n",
      "5 10 372\n",
      "5 10 373\n",
      "5 10 374\n",
      "5 10 375\n",
      "5 10 376\n",
      "5 10 377\n",
      "5 10 378\n",
      "5 10 379\n",
      "5 10 380\n",
      "5 10 381\n",
      "5 10 382\n",
      "5 10 383\n",
      "5 10 384\n",
      "5 10 385\n",
      "5 10 386\n",
      "5 10 387\n",
      "5 10 388\n",
      "5 10 389\n",
      "5 10 390\n",
      "5 10 391\n",
      "5 10 392\n",
      "5 10 393\n",
      "5 10 394\n",
      "5 10 395\n",
      "5 10 396\n",
      "5 10 397\n",
      "5 10 398\n",
      "5 10 399\n",
      "5 10 400\n",
      "5 10 401\n",
      "5 10 402\n",
      "5 10 403\n",
      "5 10 404\n",
      "5 10 405\n",
      "5 10 406\n",
      "5 10 407\n",
      "5 10 408\n",
      "5 10 409\n",
      "5 10 410\n",
      "5 10 411\n",
      "5 10 412\n",
      "5 10 413\n",
      "5 10 414\n",
      "5 10 415\n",
      "5 10 416\n",
      "5 10 417\n",
      "5 10 418\n",
      "5 10 419\n",
      "5 10 420\n",
      "5 10 421\n",
      "5 10 422\n",
      "5 10 423\n",
      "5 10 424\n",
      "5 10 425\n",
      "5 10 426\n",
      "5 10 427\n",
      "5 10 428\n",
      "5 10 429\n",
      "5 10 430\n",
      "5 10 431\n",
      "5 10 432\n",
      "5 10 433\n",
      "5 10 434\n",
      "5 10 435\n",
      "5 10 436\n",
      "5 10 437\n",
      "5 10 438\n",
      "5 10 439\n",
      "5 10 440\n",
      "5 10 441\n",
      "5 10 442\n",
      "5 10 443\n",
      "5 10 444\n",
      "5 10 445\n",
      "5 10 446\n",
      "5 10 447\n",
      "5 10 448\n",
      "5 10 449\n",
      "5 10 450\n",
      "5 10 451\n",
      "5 10 452\n",
      "5 10 453\n",
      "5 10 454\n",
      "5 10 455\n",
      "5 10 456\n",
      "5 10 457\n",
      "5 10 458\n",
      "5 10 459\n",
      "5 10 460\n",
      "5 10 461\n",
      "5 10 462\n",
      "5 10 463\n",
      "5 10 464\n",
      "5 10 465\n",
      "5 10 466\n",
      "5 10 467\n",
      "5 10 468\n",
      "5 10 469\n",
      "5 10 470\n",
      "5 10 471\n",
      "5 10 472\n",
      "5 10 473\n",
      "5 10 474\n",
      "5 10 475\n",
      "5 10 476\n",
      "5 10 477\n",
      "5 10 478\n",
      "5 10 479\n",
      "5 10 480\n",
      "5 10 481\n",
      "5 10 482\n",
      "5 10 483\n",
      "5 10 484\n",
      "5 10 485\n",
      "5 10 486\n",
      "5 10 487\n",
      "5 10 488\n",
      "5 10 489\n",
      "5 10 490\n",
      "5 10 491\n",
      "5 10 492\n",
      "5 10 493\n",
      "5 10 494\n",
      "5 10 495\n",
      "5 10 496\n",
      "5 10 497\n",
      "5 10 498\n",
      "5 10 499\n",
      "5 15 0\n",
      "5 15 1\n",
      "5 15 2\n",
      "5 15 3\n",
      "5 15 4\n",
      "5 15 5\n",
      "5 15 6\n",
      "5 15 7\n",
      "5 15 8\n",
      "5 15 9\n",
      "5 15 10\n",
      "5 15 11\n",
      "5 15 12\n",
      "5 15 13\n",
      "5 15 14\n",
      "5 15 15\n",
      "5 15 16\n",
      "5 15 17\n",
      "5 15 18\n",
      "5 15 19\n",
      "5 15 20\n",
      "5 15 21\n",
      "5 15 22\n",
      "5 15 23\n",
      "5 15 24\n",
      "5 15 25\n",
      "5 15 26\n",
      "5 15 27\n",
      "5 15 28\n",
      "5 15 29\n",
      "5 15 30\n",
      "5 15 31\n",
      "5 15 32\n",
      "5 15 33\n",
      "5 15 34\n",
      "5 15 35\n",
      "5 15 36\n",
      "5 15 37\n",
      "5 15 38\n",
      "5 15 39\n",
      "5 15 40\n",
      "5 15 41\n",
      "5 15 42\n",
      "5 15 43\n",
      "5 15 44\n",
      "5 15 45\n",
      "5 15 46\n",
      "5 15 47\n",
      "5 15 48\n",
      "5 15 49\n",
      "5 15 50\n",
      "5 15 51\n",
      "5 15 52\n",
      "5 15 53\n",
      "5 15 54\n",
      "5 15 55\n",
      "5 15 56\n",
      "5 15 57\n",
      "5 15 58\n",
      "5 15 59\n",
      "5 15 60\n",
      "5 15 61\n",
      "5 15 62\n",
      "5 15 63\n",
      "5 15 64\n",
      "5 15 65\n",
      "5 15 66\n",
      "5 15 67\n",
      "5 15 68\n",
      "5 15 69\n",
      "5 15 70\n",
      "5 15 71\n",
      "5 15 72\n",
      "5 15 73\n",
      "5 15 74\n",
      "5 15 75\n",
      "5 15 76\n",
      "5 15 77\n",
      "5 15 78\n",
      "5 15 79\n",
      "5 15 80\n",
      "5 15 81\n",
      "5 15 82\n",
      "5 15 83\n",
      "5 15 84\n",
      "5 15 85\n",
      "5 15 86\n",
      "5 15 87\n",
      "5 15 88\n",
      "5 15 89\n",
      "5 15 90\n",
      "5 15 91\n",
      "5 15 92\n",
      "5 15 93\n",
      "5 15 94\n",
      "5 15 95\n",
      "5 15 96\n",
      "5 15 97\n",
      "5 15 98\n",
      "5 15 99\n",
      "5 15 100\n",
      "5 15 101\n",
      "5 15 102\n",
      "5 15 103\n",
      "5 15 104\n",
      "5 15 105\n",
      "5 15 106\n",
      "5 15 107\n",
      "5 15 108\n",
      "5 15 109\n",
      "5 15 110\n",
      "5 15 111\n",
      "5 15 112\n",
      "5 15 113\n",
      "5 15 114\n",
      "5 15 115\n",
      "5 15 116\n",
      "5 15 117\n",
      "5 15 118\n",
      "5 15 119\n",
      "5 15 120\n",
      "5 15 121\n",
      "5 15 122\n",
      "5 15 123\n",
      "5 15 124\n",
      "5 15 125\n",
      "5 15 126\n",
      "5 15 127\n",
      "5 15 128\n",
      "5 15 129\n",
      "5 15 130\n",
      "5 15 131\n",
      "5 15 132\n",
      "5 15 133\n",
      "5 15 134\n",
      "5 15 135\n",
      "5 15 136\n",
      "5 15 137\n",
      "5 15 138\n",
      "5 15 139\n",
      "5 15 140\n",
      "5 15 141\n",
      "5 15 142\n",
      "5 15 143\n",
      "5 15 144\n",
      "5 15 145\n",
      "5 15 146\n",
      "5 15 147\n",
      "5 15 148\n",
      "5 15 149\n",
      "5 15 150\n",
      "5 15 151\n",
      "5 15 152\n",
      "5 15 153\n",
      "5 15 154\n",
      "5 15 155\n",
      "5 15 156\n",
      "5 15 157\n",
      "5 15 158\n",
      "5 15 159\n",
      "5 15 160\n",
      "5 15 161\n",
      "5 15 162\n",
      "5 15 163\n",
      "5 15 164\n",
      "5 15 165\n",
      "5 15 166\n",
      "5 15 167\n",
      "5 15 168\n",
      "5 15 169\n",
      "5 15 170\n",
      "5 15 171\n",
      "5 15 172\n",
      "5 15 173\n",
      "5 15 174\n",
      "5 15 175\n",
      "5 15 176\n",
      "5 15 177\n",
      "5 15 178\n",
      "5 15 179\n",
      "5 15 180\n",
      "5 15 181\n",
      "5 15 182\n",
      "5 15 183\n",
      "5 15 184\n",
      "5 15 185\n",
      "5 15 186\n",
      "5 15 187\n",
      "5 15 188\n",
      "5 15 189\n",
      "5 15 190\n",
      "5 15 191\n",
      "5 15 192\n",
      "5 15 193\n",
      "5 15 194\n",
      "5 15 195\n",
      "5 15 196\n",
      "5 15 197\n",
      "5 15 198\n",
      "5 15 199\n",
      "5 15 200\n",
      "5 15 201\n",
      "5 15 202\n",
      "5 15 203\n",
      "5 15 204\n",
      "5 15 205\n",
      "5 15 206\n",
      "5 15 207\n",
      "5 15 208\n",
      "5 15 209\n",
      "5 15 210\n",
      "5 15 211\n",
      "5 15 212\n",
      "5 15 213\n",
      "5 15 214\n",
      "5 15 215\n",
      "5 15 216\n",
      "5 15 217\n",
      "5 15 218\n",
      "5 15 219\n",
      "5 15 220\n",
      "5 15 221\n",
      "5 15 222\n",
      "5 15 223\n",
      "5 15 224\n",
      "5 15 225\n",
      "5 15 226\n",
      "5 15 227\n",
      "5 15 228\n",
      "5 15 229\n",
      "5 15 230\n",
      "5 15 231\n",
      "5 15 232\n",
      "5 15 233\n",
      "5 15 234\n",
      "5 15 235\n",
      "5 15 236\n",
      "5 15 237\n",
      "5 15 238\n",
      "5 15 239\n",
      "5 15 240\n",
      "5 15 241\n",
      "5 15 242\n",
      "5 15 243\n",
      "5 15 244\n",
      "5 15 245\n",
      "5 15 246\n",
      "5 15 247\n",
      "5 15 248\n",
      "5 15 249\n",
      "5 15 250\n",
      "5 15 251\n",
      "5 15 252\n",
      "5 15 253\n",
      "5 15 254\n",
      "5 15 255\n",
      "5 15 256\n",
      "5 15 257\n",
      "5 15 258\n",
      "5 15 259\n",
      "5 15 260\n",
      "5 15 261\n",
      "5 15 262\n",
      "5 15 263\n",
      "5 15 264\n",
      "5 15 265\n",
      "5 15 266\n",
      "5 15 267\n",
      "5 15 268\n",
      "5 15 269\n",
      "5 15 270\n",
      "5 15 271\n",
      "5 15 272\n",
      "5 15 273\n",
      "5 15 274\n",
      "5 15 275\n",
      "5 15 276\n",
      "5 15 277\n",
      "5 15 278\n",
      "5 15 279\n",
      "5 15 280\n",
      "5 15 281\n",
      "5 15 282\n",
      "5 15 283\n",
      "5 15 284\n",
      "5 15 285\n",
      "5 15 286\n",
      "5 15 287\n",
      "5 15 288\n",
      "5 15 289\n",
      "5 15 290\n",
      "5 15 291\n",
      "5 15 292\n",
      "5 15 293\n",
      "5 15 294\n",
      "5 15 295\n",
      "5 15 296\n",
      "5 15 297\n",
      "5 15 298\n",
      "5 15 299\n",
      "5 15 300\n",
      "5 15 301\n",
      "5 15 302\n",
      "5 15 303\n",
      "5 15 304\n",
      "5 15 305\n",
      "5 15 306\n",
      "5 15 307\n",
      "5 15 308\n",
      "5 15 309\n",
      "5 15 310\n",
      "5 15 311\n",
      "5 15 312\n",
      "5 15 313\n",
      "5 15 314\n",
      "5 15 315\n",
      "5 15 316\n",
      "5 15 317\n",
      "5 15 318\n",
      "5 15 319\n",
      "5 15 320\n",
      "5 15 321\n",
      "5 15 322\n",
      "5 15 323\n",
      "5 15 324\n",
      "5 15 325\n",
      "5 15 326\n",
      "5 15 327\n",
      "5 15 328\n",
      "5 15 329\n",
      "5 15 330\n",
      "5 15 331\n",
      "5 15 332\n",
      "5 15 333\n",
      "5 15 334\n",
      "5 15 335\n",
      "5 15 336\n",
      "5 15 337\n",
      "5 15 338\n",
      "5 15 339\n",
      "5 15 340\n",
      "5 15 341\n",
      "5 15 342\n",
      "5 15 343\n",
      "5 15 344\n",
      "5 15 345\n",
      "5 15 346\n",
      "5 15 347\n",
      "5 15 348\n",
      "5 15 349\n",
      "5 15 350\n",
      "5 15 351\n",
      "5 15 352\n",
      "5 15 353\n",
      "5 15 354\n",
      "5 15 355\n",
      "5 15 356\n",
      "5 15 357\n",
      "5 15 358\n",
      "5 15 359\n",
      "5 15 360\n",
      "5 15 361\n",
      "5 15 362\n",
      "5 15 363\n",
      "5 15 364\n",
      "5 15 365\n",
      "5 15 366\n",
      "5 15 367\n",
      "5 15 368\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "5 15 369\n",
      "5 15 370\n",
      "5 15 371\n",
      "5 15 372\n",
      "5 15 373\n",
      "5 15 374\n",
      "5 15 375\n",
      "5 15 376\n",
      "5 15 377\n",
      "5 15 378\n",
      "5 15 379\n",
      "5 15 380\n",
      "5 15 381\n",
      "5 15 382\n",
      "5 15 383\n",
      "5 15 384\n",
      "5 15 385\n",
      "5 15 386\n",
      "5 15 387\n",
      "5 15 388\n",
      "5 15 389\n",
      "5 15 390\n",
      "5 15 391\n",
      "5 15 392\n",
      "5 15 393\n",
      "5 15 394\n",
      "5 15 395\n",
      "5 15 396\n",
      "5 15 397\n",
      "5 15 398\n",
      "5 15 399\n",
      "5 15 400\n",
      "5 15 401\n",
      "5 15 402\n",
      "5 15 403\n",
      "5 15 404\n",
      "5 15 405\n",
      "5 15 406\n",
      "5 15 407\n",
      "5 15 408\n",
      "5 15 409\n",
      "5 15 410\n",
      "5 15 411\n",
      "5 15 412\n",
      "5 15 413\n",
      "5 15 414\n",
      "5 15 415\n",
      "5 15 416\n",
      "5 15 417\n",
      "5 15 418\n",
      "5 15 419\n",
      "5 15 420\n",
      "5 15 421\n",
      "5 15 422\n",
      "5 15 423\n",
      "5 15 424\n",
      "5 15 425\n",
      "5 15 426\n",
      "5 15 427\n",
      "5 15 428\n",
      "5 15 429\n",
      "5 15 430\n",
      "5 15 431\n",
      "5 15 432\n",
      "5 15 433\n",
      "5 15 434\n",
      "5 15 435\n",
      "5 15 436\n",
      "5 15 437\n",
      "5 15 438\n",
      "5 15 439\n",
      "5 15 440\n",
      "5 15 441\n",
      "5 15 442\n",
      "5 15 443\n",
      "5 15 444\n",
      "5 15 445\n",
      "5 15 446\n",
      "5 15 447\n",
      "5 15 448\n",
      "5 15 449\n",
      "5 15 450\n",
      "5 15 451\n",
      "5 15 452\n",
      "5 15 453\n",
      "5 15 454\n",
      "5 15 455\n",
      "5 15 456\n",
      "5 15 457\n",
      "5 15 458\n",
      "5 15 459\n",
      "5 15 460\n",
      "5 15 461\n",
      "5 15 462\n",
      "5 15 463\n",
      "5 15 464\n",
      "5 15 465\n",
      "5 15 466\n",
      "5 15 467\n",
      "5 15 468\n",
      "5 15 469\n",
      "5 15 470\n",
      "5 15 471\n",
      "5 15 472\n",
      "5 15 473\n",
      "5 15 474\n",
      "5 15 475\n",
      "5 15 476\n",
      "5 15 477\n",
      "5 15 478\n",
      "5 15 479\n",
      "5 15 480\n",
      "5 15 481\n",
      "5 15 482\n",
      "5 15 483\n",
      "5 15 484\n",
      "5 15 485\n",
      "5 15 486\n",
      "5 15 487\n",
      "5 15 488\n",
      "5 15 489\n",
      "5 15 490\n",
      "5 15 491\n",
      "5 15 492\n",
      "5 15 493\n",
      "5 15 494\n",
      "5 15 495\n",
      "5 15 496\n",
      "5 15 497\n",
      "5 15 498\n",
      "5 15 499\n",
      "5 20 0\n",
      "5 20 1\n",
      "5 20 2\n",
      "5 20 3\n",
      "5 20 4\n",
      "5 20 5\n",
      "5 20 6\n",
      "5 20 7\n",
      "5 20 8\n",
      "5 20 9\n",
      "5 20 10\n",
      "5 20 11\n",
      "5 20 12\n",
      "5 20 13\n",
      "5 20 14\n",
      "5 20 15\n",
      "5 20 16\n",
      "5 20 17\n",
      "5 20 18\n",
      "5 20 19\n",
      "5 20 20\n",
      "5 20 21\n",
      "5 20 22\n",
      "5 20 23\n",
      "5 20 24\n",
      "5 20 25\n",
      "5 20 26\n",
      "5 20 27\n",
      "5 20 28\n",
      "5 20 29\n",
      "5 20 30\n",
      "5 20 31\n",
      "5 20 32\n",
      "5 20 33\n",
      "5 20 34\n",
      "5 20 35\n",
      "5 20 36\n",
      "5 20 37\n",
      "5 20 38\n",
      "5 20 39\n",
      "5 20 40\n",
      "5 20 41\n",
      "5 20 42\n",
      "5 20 43\n",
      "5 20 44\n",
      "5 20 45\n",
      "5 20 46\n",
      "5 20 47\n",
      "5 20 48\n",
      "5 20 49\n",
      "5 20 50\n",
      "5 20 51\n",
      "5 20 52\n",
      "5 20 53\n",
      "5 20 54\n",
      "5 20 55\n",
      "5 20 56\n",
      "5 20 57\n",
      "5 20 58\n",
      "5 20 59\n",
      "5 20 60\n",
      "5 20 61\n",
      "5 20 62\n",
      "5 20 63\n",
      "5 20 64\n",
      "5 20 65\n",
      "5 20 66\n",
      "5 20 67\n",
      "5 20 68\n",
      "5 20 69\n",
      "5 20 70\n",
      "5 20 71\n",
      "5 20 72\n",
      "5 20 73\n",
      "5 20 74\n",
      "5 20 75\n",
      "5 20 76\n",
      "5 20 77\n",
      "5 20 78\n",
      "5 20 79\n",
      "5 20 80\n",
      "5 20 81\n",
      "5 20 82\n",
      "5 20 83\n",
      "5 20 84\n",
      "5 20 85\n",
      "5 20 86\n",
      "5 20 87\n",
      "5 20 88\n",
      "5 20 89\n",
      "5 20 90\n",
      "5 20 91\n",
      "5 20 92\n",
      "5 20 93\n",
      "5 20 94\n",
      "5 20 95\n",
      "5 20 96\n",
      "5 20 97\n",
      "5 20 98\n",
      "5 20 99\n",
      "5 20 100\n",
      "5 20 101\n",
      "5 20 102\n",
      "5 20 103\n",
      "5 20 104\n",
      "5 20 105\n",
      "5 20 106\n",
      "5 20 107\n",
      "5 20 108\n",
      "5 20 109\n",
      "5 20 110\n",
      "5 20 111\n",
      "5 20 112\n",
      "5 20 113\n",
      "5 20 114\n",
      "5 20 115\n",
      "5 20 116\n",
      "5 20 117\n",
      "5 20 118\n",
      "5 20 119\n",
      "5 20 120\n",
      "5 20 121\n",
      "5 20 122\n",
      "5 20 123\n",
      "5 20 124\n",
      "5 20 125\n",
      "5 20 126\n",
      "5 20 127\n",
      "5 20 128\n",
      "5 20 129\n",
      "5 20 130\n",
      "5 20 131\n",
      "5 20 132\n",
      "5 20 133\n",
      "5 20 134\n",
      "5 20 135\n",
      "5 20 136\n",
      "5 20 137\n",
      "5 20 138\n",
      "5 20 139\n",
      "5 20 140\n",
      "5 20 141\n",
      "5 20 142\n",
      "5 20 143\n",
      "5 20 144\n",
      "5 20 145\n",
      "5 20 146\n",
      "5 20 147\n",
      "5 20 148\n",
      "5 20 149\n",
      "5 20 150\n",
      "5 20 151\n",
      "5 20 152\n",
      "5 20 153\n",
      "5 20 154\n",
      "5 20 155\n",
      "5 20 156\n",
      "5 20 157\n",
      "5 20 158\n",
      "5 20 159\n",
      "5 20 160\n",
      "5 20 161\n",
      "5 20 162\n",
      "5 20 163\n",
      "5 20 164\n",
      "5 20 165\n",
      "5 20 166\n",
      "5 20 167\n",
      "5 20 168\n",
      "5 20 169\n",
      "5 20 170\n",
      "5 20 171\n",
      "5 20 172\n",
      "5 20 173\n",
      "5 20 174\n",
      "5 20 175\n",
      "5 20 176\n",
      "5 20 177\n",
      "5 20 178\n",
      "5 20 179\n",
      "5 20 180\n",
      "5 20 181\n",
      "5 20 182\n",
      "5 20 183\n",
      "5 20 184\n",
      "5 20 185\n",
      "5 20 186\n",
      "5 20 187\n",
      "5 20 188\n",
      "5 20 189\n",
      "5 20 190\n",
      "5 20 191\n",
      "5 20 192\n",
      "5 20 193\n",
      "5 20 194\n",
      "5 20 195\n",
      "5 20 196\n",
      "5 20 197\n",
      "5 20 198\n",
      "5 20 199\n",
      "5 20 200\n",
      "5 20 201\n",
      "5 20 202\n",
      "5 20 203\n",
      "5 20 204\n",
      "5 20 205\n",
      "5 20 206\n",
      "5 20 207\n",
      "5 20 208\n",
      "5 20 209\n",
      "5 20 210\n",
      "5 20 211\n",
      "5 20 212\n",
      "5 20 213\n",
      "5 20 214\n",
      "5 20 215\n",
      "5 20 216\n",
      "5 20 217\n",
      "5 20 218\n",
      "5 20 219\n",
      "5 20 220\n",
      "5 20 221\n",
      "5 20 222\n",
      "5 20 223\n",
      "5 20 224\n",
      "5 20 225\n",
      "5 20 226\n",
      "5 20 227\n",
      "5 20 228\n",
      "5 20 229\n",
      "5 20 230\n",
      "5 20 231\n",
      "5 20 232\n",
      "5 20 233\n",
      "5 20 234\n",
      "5 20 235\n",
      "5 20 236\n",
      "5 20 237\n",
      "5 20 238\n",
      "5 20 239\n",
      "5 20 240\n",
      "5 20 241\n",
      "5 20 242\n",
      "5 20 243\n",
      "5 20 244\n",
      "5 20 245\n",
      "5 20 246\n",
      "5 20 247\n",
      "5 20 248\n",
      "5 20 249\n",
      "5 20 250\n",
      "5 20 251\n",
      "5 20 252\n",
      "5 20 253\n",
      "5 20 254\n",
      "5 20 255\n",
      "5 20 256\n",
      "5 20 257\n",
      "5 20 258\n",
      "5 20 259\n",
      "5 20 260\n",
      "5 20 261\n",
      "5 20 262\n",
      "5 20 263\n",
      "5 20 264\n",
      "5 20 265\n",
      "5 20 266\n",
      "5 20 267\n",
      "5 20 268\n",
      "5 20 269\n",
      "5 20 270\n",
      "5 20 271\n",
      "5 20 272\n",
      "5 20 273\n",
      "5 20 274\n",
      "5 20 275\n",
      "5 20 276\n",
      "5 20 277\n",
      "5 20 278\n",
      "5 20 279\n",
      "5 20 280\n",
      "5 20 281\n",
      "5 20 282\n",
      "5 20 283\n",
      "5 20 284\n",
      "5 20 285\n",
      "5 20 286\n",
      "5 20 287\n",
      "5 20 288\n",
      "5 20 289\n",
      "5 20 290\n",
      "5 20 291\n",
      "5 20 292\n",
      "5 20 293\n",
      "5 20 294\n",
      "5 20 295\n",
      "5 20 296\n",
      "5 20 297\n",
      "5 20 298\n",
      "5 20 299\n",
      "5 20 300\n",
      "5 20 301\n",
      "5 20 302\n",
      "5 20 303\n",
      "5 20 304\n",
      "5 20 305\n",
      "5 20 306\n",
      "5 20 307\n",
      "5 20 308\n",
      "5 20 309\n",
      "5 20 310\n",
      "5 20 311\n",
      "5 20 312\n",
      "5 20 313\n",
      "5 20 314\n",
      "5 20 315\n",
      "5 20 316\n",
      "5 20 317\n",
      "5 20 318\n",
      "5 20 319\n",
      "5 20 320\n",
      "5 20 321\n",
      "5 20 322\n",
      "5 20 323\n",
      "5 20 324\n",
      "5 20 325\n",
      "5 20 326\n",
      "5 20 327\n",
      "5 20 328\n",
      "5 20 329\n",
      "5 20 330\n",
      "5 20 331\n",
      "5 20 332\n",
      "5 20 333\n",
      "5 20 334\n",
      "5 20 335\n",
      "5 20 336\n",
      "5 20 337\n",
      "5 20 338\n",
      "5 20 339\n",
      "5 20 340\n",
      "5 20 341\n",
      "5 20 342\n",
      "5 20 343\n",
      "5 20 344\n",
      "5 20 345\n",
      "5 20 346\n",
      "5 20 347\n",
      "5 20 348\n",
      "5 20 349\n",
      "5 20 350\n",
      "5 20 351\n",
      "5 20 352\n",
      "5 20 353\n",
      "5 20 354\n",
      "5 20 355\n",
      "5 20 356\n",
      "5 20 357\n",
      "5 20 358\n",
      "5 20 359\n",
      "5 20 360\n",
      "5 20 361\n",
      "5 20 362\n",
      "5 20 363\n",
      "5 20 364\n",
      "5 20 365\n",
      "5 20 366\n",
      "5 20 367\n",
      "5 20 368\n",
      "5 20 369\n",
      "5 20 370\n",
      "5 20 371\n",
      "5 20 372\n",
      "5 20 373\n",
      "5 20 374\n",
      "5 20 375\n",
      "5 20 376\n",
      "5 20 377\n",
      "5 20 378\n",
      "5 20 379\n",
      "5 20 380\n",
      "5 20 381\n",
      "5 20 382\n",
      "5 20 383\n",
      "5 20 384\n",
      "5 20 385\n",
      "5 20 386\n",
      "5 20 387\n",
      "5 20 388\n",
      "5 20 389\n",
      "5 20 390\n",
      "5 20 391\n",
      "5 20 392\n",
      "5 20 393\n",
      "5 20 394\n",
      "5 20 395\n",
      "5 20 396\n",
      "5 20 397\n",
      "5 20 398\n",
      "5 20 399\n",
      "5 20 400\n",
      "5 20 401\n",
      "5 20 402\n",
      "5 20 403\n",
      "5 20 404\n",
      "5 20 405\n",
      "5 20 406\n",
      "5 20 407\n",
      "5 20 408\n",
      "5 20 409\n",
      "5 20 410\n",
      "5 20 411\n",
      "5 20 412\n",
      "5 20 413\n",
      "5 20 414\n",
      "5 20 415\n",
      "5 20 416\n",
      "5 20 417\n",
      "5 20 418\n",
      "5 20 419\n",
      "5 20 420\n",
      "5 20 421\n",
      "5 20 422\n",
      "5 20 423\n",
      "5 20 424\n",
      "5 20 425\n",
      "5 20 426\n",
      "5 20 427\n",
      "5 20 428\n",
      "5 20 429\n",
      "5 20 430\n",
      "5 20 431\n",
      "5 20 432\n",
      "5 20 433\n",
      "5 20 434\n",
      "5 20 435\n",
      "5 20 436\n",
      "5 20 437\n",
      "5 20 438\n",
      "5 20 439\n",
      "5 20 440\n",
      "5 20 441\n",
      "5 20 442\n",
      "5 20 443\n",
      "5 20 444\n",
      "5 20 445\n",
      "5 20 446\n",
      "5 20 447\n",
      "5 20 448\n",
      "5 20 449\n",
      "5 20 450\n",
      "5 20 451\n",
      "5 20 452\n",
      "5 20 453\n",
      "5 20 454\n",
      "5 20 455\n",
      "5 20 456\n",
      "5 20 457\n",
      "5 20 458\n",
      "5 20 459\n",
      "5 20 460\n",
      "5 20 461\n",
      "5 20 462\n",
      "5 20 463\n",
      "5 20 464\n",
      "5 20 465\n",
      "5 20 466\n",
      "5 20 467\n",
      "5 20 468\n",
      "5 20 469\n",
      "5 20 470\n",
      "5 20 471\n",
      "5 20 472\n",
      "5 20 473\n",
      "5 20 474\n",
      "5 20 475\n",
      "5 20 476\n",
      "5 20 477\n",
      "5 20 478\n",
      "5 20 479\n",
      "5 20 480\n",
      "5 20 481\n",
      "5 20 482\n",
      "5 20 483\n",
      "5 20 484\n",
      "5 20 485\n",
      "5 20 486\n",
      "5 20 487\n",
      "5 20 488\n",
      "5 20 489\n",
      "5 20 490\n",
      "5 20 491\n",
      "5 20 492\n",
      "5 20 493\n",
      "5 20 494\n",
      "5 20 495\n",
      "5 20 496\n",
      "5 20 497\n",
      "5 20 498\n",
      "5 20 499\n",
      "==================== \n",
      " n= 10\n",
      "10 15 0\n",
      "10 15 1\n",
      "10 15 2\n",
      "10 15 3\n",
      "10 15 4\n",
      "10 15 5\n",
      "10 15 6\n",
      "10 15 7\n",
      "10 15 8\n",
      "10 15 9\n",
      "10 15 10\n",
      "10 15 11\n",
      "10 15 12\n",
      "10 15 13\n",
      "10 15 14\n",
      "10 15 15\n",
      "10 15 16\n",
      "10 15 17\n",
      "10 15 18\n",
      "10 15 19\n",
      "10 15 20\n",
      "10 15 21\n",
      "10 15 22\n",
      "10 15 23\n",
      "10 15 24\n",
      "10 15 25\n",
      "10 15 26\n",
      "10 15 27\n",
      "10 15 28\n",
      "10 15 29\n",
      "10 15 30\n",
      "10 15 31\n",
      "10 15 32\n",
      "10 15 33\n",
      "10 15 34\n",
      "10 15 35\n",
      "10 15 36\n",
      "10 15 37\n",
      "10 15 38\n",
      "10 15 39\n",
      "10 15 40\n",
      "10 15 41\n",
      "10 15 42\n",
      "10 15 43\n",
      "10 15 44\n",
      "10 15 45\n",
      "10 15 46\n",
      "10 15 47\n",
      "10 15 48\n",
      "10 15 49\n",
      "10 15 50\n",
      "10 15 51\n",
      "10 15 52\n",
      "10 15 53\n",
      "10 15 54\n",
      "10 15 55\n",
      "10 15 56\n",
      "10 15 57\n",
      "10 15 58\n",
      "10 15 59\n",
      "10 15 60\n",
      "10 15 61\n",
      "10 15 62\n",
      "10 15 63\n",
      "10 15 64\n",
      "10 15 65\n",
      "10 15 66\n",
      "10 15 67\n",
      "10 15 68\n",
      "10 15 69\n",
      "10 15 70\n",
      "10 15 71\n",
      "10 15 72\n",
      "10 15 73\n",
      "10 15 74\n",
      "10 15 75\n",
      "10 15 76\n",
      "10 15 77\n",
      "10 15 78\n",
      "10 15 79\n",
      "10 15 80\n",
      "10 15 81\n",
      "10 15 82\n",
      "10 15 83\n",
      "10 15 84\n",
      "10 15 85\n",
      "10 15 86\n",
      "10 15 87\n",
      "10 15 88\n",
      "10 15 89\n",
      "10 15 90\n",
      "10 15 91\n",
      "10 15 92\n",
      "10 15 93\n",
      "10 15 94\n",
      "10 15 95\n",
      "10 15 96\n",
      "10 15 97\n",
      "10 15 98\n",
      "10 15 99\n",
      "10 15 100\n",
      "10 15 101\n",
      "10 15 102\n",
      "10 15 103\n",
      "10 15 104\n",
      "10 15 105\n",
      "10 15 106\n",
      "10 15 107\n",
      "10 15 108\n",
      "10 15 109\n",
      "10 15 110\n",
      "10 15 111\n",
      "10 15 112\n",
      "10 15 113\n",
      "10 15 114\n",
      "10 15 115\n",
      "10 15 116\n",
      "10 15 117\n",
      "10 15 118\n",
      "10 15 119\n",
      "10 15 120\n",
      "10 15 121\n",
      "10 15 122\n",
      "10 15 123\n",
      "10 15 124\n",
      "10 15 125\n",
      "10 15 126\n",
      "10 15 127\n",
      "10 15 128\n",
      "10 15 129\n",
      "10 15 130\n",
      "10 15 131\n",
      "10 15 132\n",
      "10 15 133\n",
      "10 15 134\n",
      "10 15 135\n",
      "10 15 136\n",
      "10 15 137\n",
      "10 15 138\n",
      "10 15 139\n",
      "10 15 140\n",
      "10 15 141\n",
      "10 15 142\n",
      "10 15 143\n",
      "10 15 144\n",
      "10 15 145\n",
      "10 15 146\n",
      "10 15 147\n",
      "10 15 148\n",
      "10 15 149\n",
      "10 15 150\n",
      "10 15 151\n",
      "10 15 152\n",
      "10 15 153\n",
      "10 15 154\n",
      "10 15 155\n",
      "10 15 156\n",
      "10 15 157\n",
      "10 15 158\n",
      "10 15 159\n",
      "10 15 160\n",
      "10 15 161\n",
      "10 15 162\n",
      "10 15 163\n",
      "10 15 164\n",
      "10 15 165\n",
      "10 15 166\n",
      "10 15 167\n",
      "10 15 168\n",
      "10 15 169\n",
      "10 15 170\n",
      "10 15 171\n",
      "10 15 172\n",
      "10 15 173\n",
      "10 15 174\n",
      "10 15 175\n",
      "10 15 176\n",
      "10 15 177\n",
      "10 15 178\n",
      "10 15 179\n",
      "10 15 180\n",
      "10 15 181\n",
      "10 15 182\n",
      "10 15 183\n",
      "10 15 184\n",
      "10 15 185\n",
      "10 15 186\n",
      "10 15 187\n",
      "10 15 188\n",
      "10 15 189\n",
      "10 15 190\n",
      "10 15 191\n",
      "10 15 192\n",
      "10 15 193\n",
      "10 15 194\n",
      "10 15 195\n",
      "10 15 196\n",
      "10 15 197\n",
      "10 15 198\n",
      "10 15 199\n",
      "10 15 200\n",
      "10 15 201\n",
      "10 15 202\n",
      "10 15 203\n",
      "10 15 204\n",
      "10 15 205\n",
      "10 15 206\n",
      "10 15 207\n",
      "10 15 208\n",
      "10 15 209\n",
      "10 15 210\n",
      "10 15 211\n",
      "10 15 212\n",
      "10 15 213\n",
      "10 15 214\n",
      "10 15 215\n",
      "10 15 216\n",
      "10 15 217\n",
      "10 15 218\n",
      "10 15 219\n",
      "10 15 220\n",
      "10 15 221\n",
      "10 15 222\n",
      "10 15 223\n",
      "10 15 224\n",
      "10 15 225\n",
      "10 15 226\n",
      "10 15 227\n",
      "10 15 228\n",
      "10 15 229\n",
      "10 15 230\n",
      "10 15 231\n",
      "10 15 232\n",
      "10 15 233\n",
      "10 15 234\n",
      "10 15 235\n",
      "10 15 236\n",
      "10 15 237\n",
      "10 15 238\n",
      "10 15 239\n",
      "10 15 240\n",
      "10 15 241\n",
      "10 15 242\n",
      "10 15 243\n",
      "10 15 244\n",
      "10 15 245\n",
      "10 15 246\n",
      "10 15 247\n",
      "10 15 248\n",
      "10 15 249\n",
      "10 15 250\n",
      "10 15 251\n",
      "10 15 252\n",
      "10 15 253\n",
      "10 15 254\n",
      "10 15 255\n",
      "10 15 256\n",
      "10 15 257\n",
      "10 15 258\n",
      "10 15 259\n",
      "10 15 260\n",
      "10 15 261\n",
      "10 15 262\n",
      "10 15 263\n",
      "10 15 264\n",
      "10 15 265\n",
      "10 15 266\n",
      "10 15 267\n",
      "10 15 268\n",
      "10 15 269\n",
      "10 15 270\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "10 15 271\n",
      "10 15 272\n",
      "10 15 273\n",
      "10 15 274\n",
      "10 15 275\n",
      "10 15 276\n",
      "10 15 277\n",
      "10 15 278\n",
      "10 15 279\n",
      "10 15 280\n",
      "10 15 281\n",
      "10 15 282\n",
      "10 15 283\n",
      "10 15 284\n",
      "10 15 285\n",
      "10 15 286\n",
      "10 15 287\n",
      "10 15 288\n",
      "10 15 289\n",
      "10 15 290\n",
      "10 15 291\n",
      "10 15 292\n",
      "10 15 293\n",
      "10 15 294\n",
      "10 15 295\n",
      "10 15 296\n",
      "10 15 297\n",
      "10 15 298\n",
      "10 15 299\n",
      "10 15 300\n",
      "10 15 301\n",
      "10 15 302\n",
      "10 15 303\n",
      "10 15 304\n",
      "10 15 305\n",
      "10 15 306\n",
      "10 15 307\n",
      "10 15 308\n",
      "10 15 309\n",
      "10 15 310\n",
      "10 15 311\n",
      "10 15 312\n",
      "10 15 313\n",
      "10 15 314\n",
      "10 15 315\n",
      "10 15 316\n",
      "10 15 317\n",
      "10 15 318\n",
      "10 15 319\n",
      "10 15 320\n",
      "10 15 321\n",
      "10 15 322\n",
      "10 15 323\n",
      "10 15 324\n",
      "10 15 325\n",
      "10 15 326\n",
      "10 15 327\n",
      "10 15 328\n",
      "10 15 329\n",
      "10 15 330\n",
      "10 15 331\n",
      "10 15 332\n",
      "10 15 333\n",
      "10 15 334\n",
      "10 15 335\n",
      "10 15 336\n",
      "10 15 337\n",
      "10 15 338\n",
      "10 15 339\n",
      "10 15 340\n",
      "10 15 341\n",
      "10 15 342\n",
      "10 15 343\n",
      "10 15 344\n",
      "10 15 345\n",
      "10 15 346\n",
      "10 15 347\n",
      "10 15 348\n",
      "10 15 349\n",
      "10 15 350\n",
      "10 15 351\n",
      "10 15 352\n",
      "10 15 353\n",
      "10 15 354\n",
      "10 15 355\n",
      "10 15 356\n",
      "10 15 357\n",
      "10 15 358\n",
      "10 15 359\n",
      "10 15 360\n",
      "10 15 361\n",
      "10 15 362\n",
      "10 15 363\n",
      "10 15 364\n",
      "10 15 365\n",
      "10 15 366\n",
      "10 15 367\n",
      "10 15 368\n",
      "10 15 369\n",
      "10 15 370\n",
      "10 15 371\n",
      "10 15 372\n",
      "10 15 373\n",
      "10 15 374\n",
      "10 15 375\n",
      "10 15 376\n",
      "10 15 377\n",
      "10 15 378\n",
      "10 15 379\n",
      "10 15 380\n",
      "10 15 381\n",
      "10 15 382\n",
      "10 15 383\n",
      "10 15 384\n",
      "10 15 385\n",
      "10 15 386\n",
      "10 15 387\n",
      "10 15 388\n",
      "10 15 389\n",
      "10 15 390\n",
      "10 15 391\n",
      "10 15 392\n",
      "10 15 393\n",
      "10 15 394\n",
      "10 15 395\n",
      "10 15 396\n",
      "10 15 397\n",
      "10 15 398\n",
      "10 15 399\n",
      "10 15 400\n",
      "10 15 401\n",
      "10 15 402\n",
      "10 15 403\n",
      "10 15 404\n",
      "10 15 405\n",
      "10 15 406\n",
      "10 15 407\n",
      "10 15 408\n",
      "10 15 409\n",
      "10 15 410\n",
      "10 15 411\n",
      "10 15 412\n",
      "10 15 413\n",
      "10 15 414\n",
      "10 15 415\n",
      "10 15 416\n",
      "10 15 417\n",
      "10 15 418\n",
      "10 15 419\n",
      "10 15 420\n",
      "10 15 421\n",
      "10 15 422\n",
      "10 15 423\n",
      "10 15 424\n",
      "10 15 425\n",
      "10 15 426\n",
      "10 15 427\n",
      "10 15 428\n",
      "10 15 429\n",
      "10 15 430\n",
      "10 15 431\n",
      "10 15 432\n",
      "10 15 433\n",
      "10 15 434\n",
      "10 15 435\n",
      "10 15 436\n",
      "10 15 437\n",
      "10 15 438\n",
      "10 15 439\n",
      "10 15 440\n",
      "10 15 441\n",
      "10 15 442\n",
      "10 15 443\n",
      "10 15 444\n",
      "10 15 445\n",
      "10 15 446\n",
      "10 15 447\n",
      "10 15 448\n",
      "10 15 449\n",
      "10 15 450\n",
      "10 15 451\n",
      "10 15 452\n",
      "10 15 453\n",
      "10 15 454\n",
      "10 15 455\n",
      "10 15 456\n",
      "10 15 457\n",
      "10 15 458\n",
      "10 15 459\n",
      "10 15 460\n",
      "10 15 461\n",
      "10 15 462\n",
      "10 15 463\n",
      "10 15 464\n",
      "10 15 465\n",
      "10 15 466\n",
      "10 15 467\n",
      "10 15 468\n",
      "10 15 469\n",
      "10 15 470\n",
      "10 15 471\n",
      "10 15 472\n",
      "10 15 473\n",
      "10 15 474\n",
      "10 15 475\n",
      "10 15 476\n",
      "10 15 477\n",
      "10 15 478\n",
      "10 15 479\n",
      "10 15 480\n",
      "10 15 481\n",
      "10 15 482\n",
      "10 15 483\n",
      "10 15 484\n",
      "10 15 485\n",
      "10 15 486\n",
      "10 15 487\n",
      "10 15 488\n",
      "10 15 489\n",
      "10 15 490\n",
      "10 15 491\n",
      "10 15 492\n",
      "10 15 493\n",
      "10 15 494\n",
      "10 15 495\n",
      "10 15 496\n",
      "10 15 497\n",
      "10 15 498\n",
      "10 15 499\n",
      "10 20 0\n",
      "10 20 1\n",
      "10 20 2\n",
      "10 20 3\n",
      "10 20 4\n",
      "10 20 5\n",
      "10 20 6\n",
      "10 20 7\n",
      "10 20 8\n",
      "10 20 9\n",
      "10 20 10\n",
      "10 20 11\n",
      "10 20 12\n",
      "10 20 13\n",
      "10 20 14\n",
      "10 20 15\n",
      "10 20 16\n",
      "10 20 17\n",
      "10 20 18\n",
      "10 20 19\n",
      "10 20 20\n",
      "10 20 21\n",
      "10 20 22\n",
      "10 20 23\n",
      "10 20 24\n",
      "10 20 25\n",
      "10 20 26\n",
      "10 20 27\n",
      "10 20 28\n",
      "10 20 29\n",
      "10 20 30\n",
      "10 20 31\n",
      "10 20 32\n",
      "10 20 33\n",
      "10 20 34\n",
      "10 20 35\n",
      "10 20 36\n",
      "10 20 37\n",
      "10 20 38\n",
      "10 20 39\n",
      "10 20 40\n",
      "10 20 41\n",
      "10 20 42\n",
      "10 20 43\n",
      "10 20 44\n",
      "10 20 45\n",
      "10 20 46\n",
      "10 20 47\n",
      "10 20 48\n",
      "10 20 49\n",
      "10 20 50\n",
      "10 20 51\n",
      "10 20 52\n",
      "10 20 53\n",
      "10 20 54\n",
      "10 20 55\n",
      "10 20 56\n",
      "10 20 57\n",
      "10 20 58\n",
      "10 20 59\n",
      "10 20 60\n",
      "10 20 61\n",
      "10 20 62\n",
      "10 20 63\n",
      "10 20 64\n",
      "10 20 65\n",
      "10 20 66\n",
      "10 20 67\n",
      "10 20 68\n",
      "10 20 69\n",
      "10 20 70\n",
      "10 20 71\n",
      "10 20 72\n",
      "10 20 73\n",
      "10 20 74\n",
      "10 20 75\n",
      "10 20 76\n",
      "10 20 77\n",
      "10 20 78\n",
      "10 20 79\n",
      "10 20 80\n",
      "10 20 81\n",
      "10 20 82\n",
      "10 20 83\n",
      "10 20 84\n",
      "10 20 85\n",
      "10 20 86\n",
      "10 20 87\n",
      "10 20 88\n",
      "10 20 89\n",
      "10 20 90\n",
      "10 20 91\n",
      "10 20 92\n",
      "10 20 93\n",
      "10 20 94\n",
      "10 20 95\n",
      "10 20 96\n",
      "10 20 97\n",
      "10 20 98\n",
      "10 20 99\n",
      "10 20 100\n",
      "10 20 101\n",
      "10 20 102\n",
      "10 20 103\n",
      "10 20 104\n",
      "10 20 105\n",
      "10 20 106\n",
      "10 20 107\n",
      "10 20 108\n",
      "10 20 109\n",
      "10 20 110\n",
      "10 20 111\n",
      "10 20 112\n",
      "10 20 113\n",
      "10 20 114\n",
      "10 20 115\n",
      "10 20 116\n",
      "10 20 117\n",
      "10 20 118\n",
      "10 20 119\n",
      "10 20 120\n",
      "10 20 121\n",
      "10 20 122\n",
      "10 20 123\n",
      "10 20 124\n",
      "10 20 125\n",
      "10 20 126\n",
      "10 20 127\n",
      "10 20 128\n",
      "10 20 129\n",
      "10 20 130\n",
      "10 20 131\n",
      "10 20 132\n",
      "10 20 133\n",
      "10 20 134\n",
      "10 20 135\n",
      "10 20 136\n",
      "10 20 137\n",
      "10 20 138\n",
      "10 20 139\n",
      "10 20 140\n",
      "10 20 141\n",
      "10 20 142\n",
      "10 20 143\n",
      "10 20 144\n",
      "10 20 145\n",
      "10 20 146\n",
      "10 20 147\n",
      "10 20 148\n",
      "10 20 149\n",
      "10 20 150\n",
      "10 20 151\n",
      "10 20 152\n",
      "10 20 153\n",
      "10 20 154\n",
      "10 20 155\n",
      "10 20 156\n",
      "10 20 157\n",
      "10 20 158\n",
      "10 20 159\n",
      "10 20 160\n",
      "10 20 161\n",
      "10 20 162\n",
      "10 20 163\n",
      "10 20 164\n",
      "10 20 165\n",
      "10 20 166\n",
      "10 20 167\n",
      "10 20 168\n",
      "10 20 169\n",
      "10 20 170\n",
      "10 20 171\n",
      "10 20 172\n",
      "10 20 173\n",
      "10 20 174\n",
      "10 20 175\n",
      "10 20 176\n",
      "10 20 177\n",
      "10 20 178\n",
      "10 20 179\n",
      "10 20 180\n",
      "10 20 181\n",
      "10 20 182\n",
      "10 20 183\n",
      "10 20 184\n",
      "10 20 185\n",
      "10 20 186\n",
      "10 20 187\n",
      "10 20 188\n",
      "10 20 189\n",
      "10 20 190\n",
      "10 20 191\n",
      "10 20 192\n",
      "10 20 193\n",
      "10 20 194\n",
      "10 20 195\n",
      "10 20 196\n",
      "10 20 197\n",
      "10 20 198\n",
      "10 20 199\n",
      "10 20 200\n",
      "10 20 201\n",
      "10 20 202\n",
      "10 20 203\n",
      "10 20 204\n",
      "10 20 205\n",
      "10 20 206\n",
      "10 20 207\n",
      "10 20 208\n",
      "10 20 209\n",
      "10 20 210\n",
      "10 20 211\n",
      "10 20 212\n",
      "10 20 213\n",
      "10 20 214\n",
      "10 20 215\n",
      "10 20 216\n",
      "10 20 217\n",
      "10 20 218\n",
      "10 20 219\n",
      "10 20 220\n",
      "10 20 221\n",
      "10 20 222\n",
      "10 20 223\n",
      "10 20 224\n",
      "10 20 225\n",
      "10 20 226\n",
      "10 20 227\n",
      "10 20 228\n",
      "10 20 229\n",
      "10 20 230\n",
      "10 20 231\n",
      "10 20 232\n",
      "10 20 233\n",
      "10 20 234\n",
      "10 20 235\n",
      "10 20 236\n",
      "10 20 237\n",
      "10 20 238\n",
      "10 20 239\n",
      "10 20 240\n",
      "10 20 241\n",
      "10 20 242\n",
      "10 20 243\n",
      "10 20 244\n",
      "10 20 245\n",
      "10 20 246\n",
      "10 20 247\n",
      "10 20 248\n",
      "10 20 249\n",
      "10 20 250\n",
      "10 20 251\n",
      "10 20 252\n",
      "10 20 253\n",
      "10 20 254\n",
      "10 20 255\n",
      "10 20 256\n",
      "10 20 257\n",
      "10 20 258\n",
      "10 20 259\n",
      "10 20 260\n",
      "10 20 261\n",
      "10 20 262\n",
      "10 20 263\n",
      "10 20 264\n",
      "10 20 265\n",
      "10 20 266\n",
      "10 20 267\n",
      "10 20 268\n",
      "10 20 269\n",
      "10 20 270\n",
      "10 20 271\n",
      "10 20 272\n",
      "10 20 273\n",
      "10 20 274\n",
      "10 20 275\n",
      "10 20 276\n",
      "10 20 277\n",
      "10 20 278\n",
      "10 20 279\n",
      "10 20 280\n",
      "10 20 281\n",
      "10 20 282\n",
      "10 20 283\n",
      "10 20 284\n",
      "10 20 285\n",
      "10 20 286\n",
      "10 20 287\n",
      "10 20 288\n",
      "10 20 289\n",
      "10 20 290\n",
      "10 20 291\n",
      "10 20 292\n",
      "10 20 293\n",
      "10 20 294\n",
      "10 20 295\n",
      "10 20 296\n",
      "10 20 297\n",
      "10 20 298\n",
      "10 20 299\n",
      "10 20 300\n",
      "10 20 301\n",
      "10 20 302\n",
      "10 20 303\n",
      "10 20 304\n",
      "10 20 305\n",
      "10 20 306\n",
      "10 20 307\n",
      "10 20 308\n",
      "10 20 309\n",
      "10 20 310\n",
      "10 20 311\n",
      "10 20 312\n",
      "10 20 313\n",
      "10 20 314\n",
      "10 20 315\n",
      "10 20 316\n",
      "10 20 317\n",
      "10 20 318\n",
      "10 20 319\n",
      "10 20 320\n",
      "10 20 321\n",
      "10 20 322\n",
      "10 20 323\n",
      "10 20 324\n",
      "10 20 325\n",
      "10 20 326\n",
      "10 20 327\n",
      "10 20 328\n",
      "10 20 329\n",
      "10 20 330\n",
      "10 20 331\n",
      "10 20 332\n",
      "10 20 333\n",
      "10 20 334\n",
      "10 20 335\n",
      "10 20 336\n",
      "10 20 337\n",
      "10 20 338\n",
      "10 20 339\n",
      "10 20 340\n",
      "10 20 341\n",
      "10 20 342\n",
      "10 20 343\n",
      "10 20 344\n",
      "10 20 345\n",
      "10 20 346\n",
      "10 20 347\n",
      "10 20 348\n",
      "10 20 349\n",
      "10 20 350\n",
      "10 20 351\n",
      "10 20 352\n",
      "10 20 353\n",
      "10 20 354\n",
      "10 20 355\n",
      "10 20 356\n",
      "10 20 357\n",
      "10 20 358\n",
      "10 20 359\n",
      "10 20 360\n",
      "10 20 361\n",
      "10 20 362\n",
      "10 20 363\n",
      "10 20 364\n",
      "10 20 365\n",
      "10 20 366\n",
      "10 20 367\n",
      "10 20 368\n",
      "10 20 369\n",
      "10 20 370\n",
      "10 20 371\n",
      "10 20 372\n",
      "10 20 373\n",
      "10 20 374\n",
      "10 20 375\n",
      "10 20 376\n",
      "10 20 377\n",
      "10 20 378\n",
      "10 20 379\n",
      "10 20 380\n",
      "10 20 381\n",
      "10 20 382\n",
      "10 20 383\n",
      "10 20 384\n",
      "10 20 385\n",
      "10 20 386\n",
      "10 20 387\n",
      "10 20 388\n",
      "10 20 389\n",
      "10 20 390\n",
      "10 20 391\n",
      "10 20 392\n",
      "10 20 393\n",
      "10 20 394\n",
      "10 20 395\n",
      "10 20 396\n",
      "10 20 397\n",
      "10 20 398\n",
      "10 20 399\n",
      "10 20 400\n",
      "10 20 401\n",
      "10 20 402\n",
      "10 20 403\n",
      "10 20 404\n",
      "10 20 405\n",
      "10 20 406\n",
      "10 20 407\n",
      "10 20 408\n",
      "10 20 409\n",
      "10 20 410\n",
      "10 20 411\n",
      "10 20 412\n",
      "10 20 413\n",
      "10 20 414\n",
      "10 20 415\n",
      "10 20 416\n",
      "10 20 417\n",
      "10 20 418\n",
      "10 20 419\n",
      "10 20 420\n",
      "10 20 421\n",
      "10 20 422\n",
      "10 20 423\n",
      "10 20 424\n",
      "10 20 425\n",
      "10 20 426\n",
      "10 20 427\n",
      "10 20 428\n",
      "10 20 429\n",
      "10 20 430\n",
      "10 20 431\n",
      "10 20 432\n",
      "10 20 433\n",
      "10 20 434\n",
      "10 20 435\n",
      "10 20 436\n",
      "10 20 437\n",
      "10 20 438\n",
      "10 20 439\n",
      "10 20 440\n",
      "10 20 441\n",
      "10 20 442\n",
      "10 20 443\n",
      "10 20 444\n",
      "10 20 445\n",
      "10 20 446\n",
      "10 20 447\n",
      "10 20 448\n",
      "10 20 449\n",
      "10 20 450\n",
      "10 20 451\n",
      "10 20 452\n",
      "10 20 453\n",
      "10 20 454\n",
      "10 20 455\n",
      "10 20 456\n",
      "10 20 457\n",
      "10 20 458\n",
      "10 20 459\n",
      "10 20 460\n",
      "10 20 461\n",
      "10 20 462\n",
      "10 20 463\n",
      "10 20 464\n",
      "10 20 465\n",
      "10 20 466\n",
      "10 20 467\n",
      "10 20 468\n",
      "10 20 469\n",
      "10 20 470\n",
      "10 20 471\n",
      "10 20 472\n",
      "10 20 473\n",
      "10 20 474\n",
      "10 20 475\n",
      "10 20 476\n",
      "10 20 477\n",
      "10 20 478\n",
      "10 20 479\n",
      "10 20 480\n",
      "10 20 481\n",
      "10 20 482\n",
      "10 20 483\n",
      "10 20 484\n",
      "10 20 485\n",
      "10 20 486\n",
      "10 20 487\n",
      "10 20 488\n",
      "10 20 489\n",
      "10 20 490\n",
      "10 20 491\n",
      "10 20 492\n",
      "10 20 493\n",
      "10 20 494\n",
      "10 20 495\n",
      "10 20 496\n",
      "10 20 497\n",
      "10 20 498\n",
      "10 20 499\n",
      "==================== \n",
      " n= 15\n",
      "15 20 0\n",
      "15 20 1\n",
      "15 20 2\n",
      "15 20 3\n",
      "15 20 4\n",
      "15 20 5\n",
      "15 20 6\n",
      "15 20 7\n",
      "15 20 8\n",
      "15 20 9\n",
      "15 20 10\n",
      "15 20 11\n",
      "15 20 12\n",
      "15 20 13\n",
      "15 20 14\n",
      "15 20 15\n",
      "15 20 16\n",
      "15 20 17\n",
      "15 20 18\n",
      "15 20 19\n",
      "15 20 20\n",
      "15 20 21\n",
      "15 20 22\n",
      "15 20 23\n",
      "15 20 24\n",
      "15 20 25\n",
      "15 20 26\n",
      "15 20 27\n",
      "15 20 28\n",
      "15 20 29\n",
      "15 20 30\n",
      "15 20 31\n",
      "15 20 32\n",
      "15 20 33\n",
      "15 20 34\n",
      "15 20 35\n",
      "15 20 36\n",
      "15 20 37\n",
      "15 20 38\n",
      "15 20 39\n",
      "15 20 40\n",
      "15 20 41\n",
      "15 20 42\n",
      "15 20 43\n",
      "15 20 44\n",
      "15 20 45\n",
      "15 20 46\n",
      "15 20 47\n",
      "15 20 48\n",
      "15 20 49\n",
      "15 20 50\n",
      "15 20 51\n",
      "15 20 52\n",
      "15 20 53\n",
      "15 20 54\n",
      "15 20 55\n",
      "15 20 56\n",
      "15 20 57\n",
      "15 20 58\n",
      "15 20 59\n",
      "15 20 60\n",
      "15 20 61\n",
      "15 20 62\n",
      "15 20 63\n",
      "15 20 64\n",
      "15 20 65\n",
      "15 20 66\n",
      "15 20 67\n",
      "15 20 68\n",
      "15 20 69\n",
      "15 20 70\n",
      "15 20 71\n",
      "15 20 72\n",
      "15 20 73\n",
      "15 20 74\n",
      "15 20 75\n",
      "15 20 76\n",
      "15 20 77\n",
      "15 20 78\n",
      "15 20 79\n",
      "15 20 80\n",
      "15 20 81\n",
      "15 20 82\n",
      "15 20 83\n",
      "15 20 84\n",
      "15 20 85\n",
      "15 20 86\n",
      "15 20 87\n",
      "15 20 88\n",
      "15 20 89\n",
      "15 20 90\n",
      "15 20 91\n",
      "15 20 92\n",
      "15 20 93\n",
      "15 20 94\n",
      "15 20 95\n",
      "15 20 96\n",
      "15 20 97\n",
      "15 20 98\n",
      "15 20 99\n",
      "15 20 100\n",
      "15 20 101\n",
      "15 20 102\n",
      "15 20 103\n",
      "15 20 104\n",
      "15 20 105\n",
      "15 20 106\n",
      "15 20 107\n",
      "15 20 108\n",
      "15 20 109\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "15 20 110\n",
      "15 20 111\n",
      "15 20 112\n",
      "15 20 113\n",
      "15 20 114\n",
      "15 20 115\n",
      "15 20 116\n",
      "15 20 117\n",
      "15 20 118\n",
      "15 20 119\n",
      "15 20 120\n",
      "15 20 121\n",
      "15 20 122\n",
      "15 20 123\n",
      "15 20 124\n",
      "15 20 125\n",
      "15 20 126\n",
      "15 20 127\n",
      "15 20 128\n",
      "15 20 129\n",
      "15 20 130\n",
      "15 20 131\n",
      "15 20 132\n",
      "15 20 133\n",
      "15 20 134\n",
      "15 20 135\n",
      "15 20 136\n",
      "15 20 137\n",
      "15 20 138\n",
      "15 20 139\n",
      "15 20 140\n",
      "15 20 141\n",
      "15 20 142\n",
      "15 20 143\n",
      "15 20 144\n",
      "15 20 145\n",
      "15 20 146\n",
      "15 20 147\n",
      "15 20 148\n",
      "15 20 149\n",
      "15 20 150\n",
      "15 20 151\n",
      "15 20 152\n",
      "15 20 153\n",
      "15 20 154\n",
      "15 20 155\n",
      "15 20 156\n",
      "15 20 157\n",
      "15 20 158\n",
      "15 20 159\n",
      "15 20 160\n",
      "15 20 161\n",
      "15 20 162\n",
      "15 20 163\n",
      "15 20 164\n",
      "15 20 165\n",
      "15 20 166\n",
      "15 20 167\n",
      "15 20 168\n",
      "15 20 169\n",
      "15 20 170\n",
      "15 20 171\n",
      "15 20 172\n",
      "15 20 173\n",
      "15 20 174\n",
      "15 20 175\n",
      "15 20 176\n",
      "15 20 177\n",
      "15 20 178\n",
      "15 20 179\n",
      "15 20 180\n",
      "15 20 181\n",
      "15 20 182\n",
      "15 20 183\n",
      "15 20 184\n",
      "15 20 185\n",
      "15 20 186\n",
      "15 20 187\n",
      "15 20 188\n",
      "15 20 189\n",
      "15 20 190\n",
      "15 20 191\n",
      "15 20 192\n",
      "15 20 193\n",
      "15 20 194\n",
      "15 20 195\n",
      "15 20 196\n",
      "15 20 197\n",
      "15 20 198\n",
      "15 20 199\n",
      "15 20 200\n",
      "15 20 201\n",
      "15 20 202\n",
      "15 20 203\n",
      "15 20 204\n",
      "15 20 205\n",
      "15 20 206\n",
      "15 20 207\n",
      "15 20 208\n",
      "15 20 209\n",
      "15 20 210\n",
      "15 20 211\n",
      "15 20 212\n",
      "15 20 213\n",
      "15 20 214\n",
      "15 20 215\n",
      "15 20 216\n",
      "15 20 217\n",
      "15 20 218\n",
      "15 20 219\n",
      "15 20 220\n",
      "15 20 221\n",
      "15 20 222\n",
      "15 20 223\n",
      "15 20 224\n",
      "15 20 225\n",
      "15 20 226\n",
      "15 20 227\n",
      "15 20 228\n",
      "15 20 229\n",
      "15 20 230\n",
      "15 20 231\n",
      "15 20 232\n",
      "15 20 233\n",
      "15 20 234\n",
      "15 20 235\n",
      "15 20 236\n",
      "15 20 237\n",
      "15 20 238\n",
      "15 20 239\n",
      "15 20 240\n",
      "15 20 241\n",
      "15 20 242\n",
      "15 20 243\n",
      "15 20 244\n",
      "15 20 245\n",
      "15 20 246\n",
      "15 20 247\n",
      "15 20 248\n",
      "15 20 249\n",
      "15 20 250\n",
      "15 20 251\n",
      "15 20 252\n",
      "15 20 253\n",
      "15 20 254\n",
      "15 20 255\n",
      "15 20 256\n",
      "15 20 257\n",
      "15 20 258\n",
      "15 20 259\n",
      "15 20 260\n",
      "15 20 261\n",
      "15 20 262\n",
      "15 20 263\n",
      "15 20 264\n",
      "15 20 265\n",
      "15 20 266\n",
      "15 20 267\n",
      "15 20 268\n",
      "15 20 269\n",
      "15 20 270\n",
      "15 20 271\n",
      "15 20 272\n",
      "15 20 273\n",
      "15 20 274\n",
      "15 20 275\n",
      "15 20 276\n",
      "15 20 277\n",
      "15 20 278\n",
      "15 20 279\n",
      "15 20 280\n",
      "15 20 281\n",
      "15 20 282\n",
      "15 20 283\n",
      "15 20 284\n",
      "15 20 285\n",
      "15 20 286\n",
      "15 20 287\n",
      "15 20 288\n",
      "15 20 289\n",
      "15 20 290\n",
      "15 20 291\n",
      "15 20 292\n",
      "15 20 293\n",
      "15 20 294\n",
      "15 20 295\n",
      "15 20 296\n",
      "15 20 297\n",
      "15 20 298\n",
      "15 20 299\n",
      "15 20 300\n",
      "15 20 301\n",
      "15 20 302\n",
      "15 20 303\n",
      "15 20 304\n",
      "15 20 305\n",
      "15 20 306\n",
      "15 20 307\n",
      "15 20 308\n",
      "15 20 309\n",
      "15 20 310\n",
      "15 20 311\n",
      "15 20 312\n",
      "15 20 313\n",
      "15 20 314\n",
      "15 20 315\n",
      "15 20 316\n",
      "15 20 317\n",
      "15 20 318\n",
      "15 20 319\n",
      "15 20 320\n",
      "15 20 321\n",
      "15 20 322\n",
      "15 20 323\n",
      "15 20 324\n",
      "15 20 325\n",
      "15 20 326\n",
      "15 20 327\n",
      "15 20 328\n",
      "15 20 329\n",
      "15 20 330\n",
      "15 20 331\n",
      "15 20 332\n",
      "15 20 333\n",
      "15 20 334\n",
      "15 20 335\n",
      "15 20 336\n",
      "15 20 337\n",
      "15 20 338\n",
      "15 20 339\n",
      "15 20 340\n",
      "15 20 341\n",
      "15 20 342\n",
      "15 20 343\n",
      "15 20 344\n",
      "15 20 345\n",
      "15 20 346\n",
      "15 20 347\n",
      "15 20 348\n",
      "15 20 349\n",
      "15 20 350\n",
      "15 20 351\n",
      "15 20 352\n",
      "15 20 353\n",
      "15 20 354\n",
      "15 20 355\n",
      "15 20 356\n",
      "15 20 357\n",
      "15 20 358\n",
      "15 20 359\n",
      "15 20 360\n",
      "15 20 361\n",
      "15 20 362\n",
      "15 20 363\n",
      "15 20 364\n",
      "15 20 365\n",
      "15 20 366\n",
      "15 20 367\n",
      "15 20 368\n",
      "15 20 369\n",
      "15 20 370\n",
      "15 20 371\n",
      "15 20 372\n",
      "15 20 373\n",
      "15 20 374\n",
      "15 20 375\n",
      "15 20 376\n",
      "15 20 377\n",
      "15 20 378\n",
      "15 20 379\n",
      "15 20 380\n",
      "15 20 381\n",
      "15 20 382\n",
      "15 20 383\n",
      "15 20 384\n",
      "15 20 385\n",
      "15 20 386\n",
      "15 20 387\n",
      "15 20 388\n",
      "15 20 389\n",
      "15 20 390\n",
      "15 20 391\n",
      "15 20 392\n",
      "15 20 393\n",
      "15 20 394\n",
      "15 20 395\n",
      "15 20 396\n",
      "15 20 397\n",
      "15 20 398\n",
      "15 20 399\n",
      "15 20 400\n",
      "15 20 401\n",
      "15 20 402\n",
      "15 20 403\n",
      "15 20 404\n",
      "15 20 405\n",
      "15 20 406\n",
      "15 20 407\n",
      "15 20 408\n",
      "15 20 409\n",
      "15 20 410\n",
      "15 20 411\n",
      "15 20 412\n",
      "15 20 413\n",
      "15 20 414\n",
      "15 20 415\n",
      "15 20 416\n",
      "15 20 417\n",
      "15 20 418\n",
      "15 20 419\n",
      "15 20 420\n",
      "15 20 421\n",
      "15 20 422\n",
      "15 20 423\n",
      "15 20 424\n",
      "15 20 425\n",
      "15 20 426\n",
      "15 20 427\n",
      "15 20 428\n",
      "15 20 429\n",
      "15 20 430\n",
      "15 20 431\n",
      "15 20 432\n",
      "15 20 433\n",
      "15 20 434\n",
      "15 20 435\n",
      "15 20 436\n",
      "15 20 437\n",
      "15 20 438\n",
      "15 20 439\n",
      "15 20 440\n",
      "15 20 441\n",
      "15 20 442\n",
      "15 20 443\n",
      "15 20 444\n",
      "15 20 445\n",
      "15 20 446\n",
      "15 20 447\n",
      "15 20 448\n",
      "15 20 449\n",
      "15 20 450\n",
      "15 20 451\n",
      "15 20 452\n",
      "15 20 453\n",
      "15 20 454\n",
      "15 20 455\n",
      "15 20 456\n",
      "15 20 457\n",
      "15 20 458\n",
      "15 20 459\n",
      "15 20 460\n",
      "15 20 461\n",
      "15 20 462\n",
      "15 20 463\n",
      "15 20 464\n",
      "15 20 465\n",
      "15 20 466\n",
      "15 20 467\n",
      "15 20 468\n",
      "15 20 469\n",
      "15 20 470\n",
      "15 20 471\n",
      "15 20 472\n",
      "15 20 473\n",
      "15 20 474\n",
      "15 20 475\n",
      "15 20 476\n",
      "15 20 477\n",
      "15 20 478\n",
      "15 20 479\n",
      "15 20 480\n",
      "15 20 481\n",
      "15 20 482\n",
      "15 20 483\n",
      "15 20 484\n",
      "15 20 485\n",
      "15 20 486\n",
      "15 20 487\n",
      "15 20 488\n",
      "15 20 489\n",
      "15 20 490\n",
      "15 20 491\n",
      "15 20 492\n",
      "15 20 493\n",
      "15 20 494\n",
      "15 20 495\n",
      "15 20 496\n",
      "15 20 497\n",
      "15 20 498\n",
      "15 20 499\n",
      "Done\n"
     ]
    }
   ],
   "source": [
    "N,n_max=500,20\n",
    "D=10 # Percent to report\n",
    "k,j=0,0 # Counter\n",
    "d,j={},0\n",
    "Q_o={ 3:[5,10,15,20], 5:[10,15,20], 10:[15,20], 15:[20] }\n",
    "for n in Q_o.keys():\n",
    "    print(\"=\"*20,\"\\n\",\"n=\",n)\n",
    "    for q_o in Q_o[n]:\n",
    "        d[n,q_o]=np.zeros(N)\n",
    "        for i in range(N):\n",
    "            j+=1\n",
    "            print(n,q_o,i)\n",
    "#             q_i=np.random.randint(n,q_o+1)\n",
    "            z_i=pp.zonotope(G=np.random.normal(size=(n,n)))\n",
    "            z_o=pp.zonotope(G=np.random.normal(size=(n,q_o)))\n",
    "#             x=time.time()\n",
    "            a_suf=alpha_sufficient(z_i,z_o)\n",
    "#             print(\"sufficient:\",time.time()-x)\n",
    "#             x=time.time()\n",
    "            a_nec=alpha_necessary(z_i,z_o)\n",
    "#             print(\"necessary:\",time.time()-x)\n",
    "            d[n,q_o][i]=1-a_suf/a_nec\n",
    "#             print(\"gap:\",n,q_o,\"=\",d[n,q_o][i])\n",
    "            j+=1\n",
    "print(\"Done\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 33,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "(3, 5) [496, 496, 500, 500] 0.049187775432405045\n",
      "(3, 10) [496, 499, 500, 500] 0.04621478578855909\n",
      "(3, 15) [496, 499, 500, 500] 0.025488967420588482\n",
      "(3, 20) [500, 500, 500, 500] 0.0008762382316735984\n",
      "(5, 10) [476, 489, 500, 500] 0.02700849729590582\n",
      "(5, 15) [472, 490, 500, 500] 0.04464416234485524\n",
      "(5, 20) [480, 493, 500, 500] 0.04962959408652312\n",
      "(10, 15) [421, 453, 492, 500] 0.07280827975640214\n",
      "(10, 20) [332, 381, 474, 499] 0.10066627720859611\n",
      "(15, 20) [370, 420, 489, 500] 0.08882584056299803\n"
     ]
    }
   ],
   "source": [
    "T=[0.001,0.01,0.05,0.1]\n",
    "for key in d.keys():\n",
    "    print(key,[np.sum(d[key]<t) for t in T],np.max(d[key]))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 35,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "10000"
      ]
     },
     "execution_count": 35,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "j"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


NameError: name 'null' is not defined